In [1]:
from google.colab import drive
drive.mount('/content/drive')
%cd '/content/drive/Shared drives/Fall2020DSGirlPower/SI650/project/DATA/TO_data/'

Mounted at /content/drive
/content/drive/Shared drives/Fall2020DSGirlPower/SI650/project/DATA/TO_data


In [2]:
import pandas as pd
import re
from nltk.corpus import stopwords
import nltk
nltk.download('stopwords')
from nltk.tokenize import word_tokenize 
nltk.download('punkt')

from gensim.models import Word2Vec

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [3]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import random
from nltk.stem.snowball import SnowballStemmer

### Load Data

In [4]:
# If it is the first run, run this
reviews = pd.read_csv("reviews_TO_act.csv")
bus = pd.read_csv("business.csv")

In [ ]:
# Read already processed data
ReviewSummary = pd.read_csv("input_data_cat.csv")

In [ ]:
reviews.head()

,Unnamed: 0,user_id,business_id,rating,date,text
0,25,TZQSUDDcA4ek5gBd6BzcjA,qUWqjjjfpB2-4P3He5rsKw,4.0,2017-06-06 19:04:10,"In the heart of Chinatown, I discovered it enr..."
1,53,AdMCAKmNDIaubYPxnjTq1g,nIIuxIn5RU0j-XkDWh3hlA,1.0,2018-10-04 14:12:57,Passed by here after we went to raijin ramen a...
2,101,1kNsEAhGU8d8xugMuXJGFA,cDoo0Pf1d_P79Rq3ZNk-Mw,3.0,2013-07-13 00:57:45,"Sigh. This review was tough to give, as it cou..."
3,114,Eg_VTAQwK3CxYTORNRWd2g,4m_hApwQ054v3ue_OxFmGw,5.0,2013-02-21 23:37:18,"Excellent food, service and ambiance! The tapa..."
4,148,RRhco4irDXPk5EJmvuwr2w,PMPMHHLe9vFzcR67iCjgLA,5.0,2016-01-17 05:23:59,Been to this a couple of times. Excellent food...


In [ ]:
bus.head(3)

,Unnamed: 0,business_id,name,address,city,state,postal_code,latitude,longitude,rating,review_count,is_open,attributes,categories,hours
0,9,EosRKXIGeSWFYWwpkbhNnA,Xtreme Couture,700 Kipling Avenue Etobicoke,Toronto,ON,M8Z 5G3,43.624539,-79.529108,3.0,16,1,"{'GoodForKids': 'True', 'ByAppointmentOnly': '...","Martial Arts, Gyms, Fitness & Instruction, Act...","{'Monday': '5:30-23:0', 'Tuesday': '5:30-23:0'..."
1,26,1wWneWD_E1pBIyVpdHMaQg,Air Jordan Store,306 Yonge Street,Toronto,ON,M5B 1R4,43.656542,-79.381308,4.0,9,1,"{'RestaurantsPriceRange2': '3', 'BikeParking':...","Shopping, Shoe Stores, Fashion","{'Monday': '10:0-21:0', 'Tuesday': '10:0-21:0'..."
2,29,007Dg4ESDVacWcC4Vq704Q,Front Door Organics,415 Horner Avenue,Toronto,ON,M8W 4W3,43.603232,-79.538424,4.0,8,0,"{'RestaurantsPriceRange2': '3', 'BikeParking':...","Shopping, Food, Organic Stores, Specialty Food...","{'Monday': '9:0-16:0', 'Tuesday': '9:0-16:0', ..."


In [5]:
# Drop unrelated columns
del reviews['Unnamed: 0']
del bus['Unnamed: 0']

In [ ]:
reviews.columns

Index(['user_id', 'business_id', 'rating', 'date', 'text'], dtype='object')

In [ ]:
bus.columns

Index(['business_id', 'name', 'address', 'city', 'state', 'postal_code',
       'latitude', 'longitude', 'rating', 'review_count', 'is_open',
       'attributes', 'categories', 'hours'],
      dtype='object')

In [6]:
bus.head(1).categories.values

array(['Martial Arts, Gyms, Fitness & Instruction, Active Life'],
      dtype=object)

In [ ]:
bus.iloc[0].categories

'Martial Arts, Gyms, Fitness & Instruction, Active Life'

### Filter Data

In [14]:
mask_restaurants = bus["categories"].str.contains("Restaurants")
mask_food = bus["categories"].str.contains("Food")
bus = bus[mask_restaurants&mask_food]

In [15]:
bus

,business_id,name,address,city,state,postal_code,latitude,longitude,rating,review_count,is_open,attributes,categories,hours
3,rVBPQdeayMYht4Uv_FOLHg,Gourmet Burger Company,843 Kipling Avenue,Toronto,ON,M8Z 5G9,43.633291,-79.531768,3.0,13,0,"{'RestaurantsPriceRange2': '2', 'RestaurantsAt...","Restaurants, Burgers, Food",NaN
11,8k62wYhDVq1-652YbJi5eg,Tim Hortons,90 Adelaide Street W,Toronto,ON,M5H 3V9,43.649859,-79.382060,3.0,8,1,"{'OutdoorSeating': 'False', 'RestaurantsDelive...","Bagels, Donuts, Food, Cafes, Coffee & Tea, Res...",NaN
14,0DnQh8SE8BSnvJltGCCiWg,Chick-N-Joy,3-1265 York Mills Road,Toronto,ON,M3A 1Z3,43.765279,-79.326248,3.0,11,1,"{'NoiseLevel': ""'loud'"", 'BusinessParking': ""{...","Fast Food, Restaurants, Chicken Shop",NaN
15,NLaK58WvlNQdUunSIkt-jA,Zav Coffee Shop & Gallery,2048 Danforth Avenue,Toronto,ON,M4C 1J6,43.685608,-79.313936,4.5,24,1,"{'DogsAllowed': 'False', 'OutdoorSeating': 'Tr...","Coffee & Tea, Restaurants, Sandwiches, Food","{'Monday': '0:0-0:0', 'Tuesday': '7:30-17:0', ..."
18,0nqQIM2PvOcTX6v0L537_Q,Fearless Meat,884 Kingston Road,Toronto,ON,M4E 1S3,43.680413,-79.290279,4.0,17,1,"{'HasTV': 'False', 'DogsAllowed': 'False', 'Bu...","Fast Food, Burgers, Sandwiches, Restaurants","{'Monday': '0:0-0:0', 'Tuesday': '8:0-20:0', '..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20333,AsvquIiLiKuXnGq2Br0ojA,Kupfert & Kim,100 Bloor Street W,Toronto,ON,M5S 3L7,43.669798,-79.391163,4.5,5,1,"{'Ambience': ""{'touristy': False, 'hipster': F...","Gluten-Free, Food, Vegetarian, Juice Bars & Sm...","{'Monday': '10:0-20:0', 'Tuesday': '10:0-20:0'..."
20337,lkq6i2x3vUsR7ZNrIFqoIw,The Monks Kettle,3073 Bloor Street W,Toronto,ON,M8X,43.646814,-79.513834,3.5,31,1,"{'OutdoorSeating': 'False', 'NoiseLevel': ""u'a...","Nightlife, Bars, Gastropubs, Restaurants, Beer...","{'Monday': '14:0-0:0', 'Tuesday': '14:0-0:0', ..."
20344,YLzQJESNAPjizXMQu4FlPg,Lick's Homeburgers and Ice cream,860 York Mills Rd,Toronto,ON,M3B 1Y4,43.754765,-79.352043,2.5,3,0,"{'Alcohol': ""u'none'""}","Restaurants, Burgers, Food, Ice Cream & Frozen...",NaN
20350,2auigv5pF08RjJ8CL1dIVw,O&B Canteen,330 King Street W,Toronto,ON,M5V 3X2,43.646693,-79.390210,3.0,291,1,"{'RestaurantsGoodForGroups': 'True', 'Alcohol'...","American (New), Restaurants, Canadian (New), C...","{'Monday': '8:0-21:0', 'Tuesday': '8:0-22:0', ..."


In [16]:
bus_id = set(rf_bus.business_id)
# filter reveiw data
reviews = reviews[reviews["business_id"].isin(bus_id)]
reviews

,user_id,business_id,rating,date,text
4,RRhco4irDXPk5EJmvuwr2w,PMPMHHLe9vFzcR67iCjgLA,5.0,2016-01-17 05:23:59,Been to this a couple of times. Excellent food...
9,pMefTWo6gMdx8WhYSA2u3w,PjtX-5vSTBVyWfMREX8bRA,4.0,2013-11-14 00:57:42,I'm so sorry to do this to you Capitano and wh...
11,BwwqlPVsJk1dbUyNwCWB7Q,28adZ4lsuUeVB2aWzohK9g,5.0,2017-11-06 22:08:35,Amazing place! My co-workers and I went here o...
13,kmOvnwtroMBC8y9lOR29bg,tvYID0arhN-shKGUrC1Wsg,1.0,2013-08-03 20:59:19,My wife is Brazilian and wanted to show me Bra...
14,0aHFybE0id9DD97QzAfNDQ,sSdHUsUZ4mvkb5ymIOJ_Aw,4.0,2016-11-10 02:00:55,I had a durian mille crepe cake and the raindr...
...,...,...,...,...,...
465361,nvPEO_zNLmNa1HW5sar7fA,9tfw-OEfpF0qC2hSzRks6g,3.0,2019-11-24 00:18:59,Location is great. Space is small and tables t...
465363,v-HcprOKPC1_F5_15t4JDw,G2Z-4U6cDsDQvT6v3FzYmA,3.0,2019-10-04 15:46:59,We were here for dinner and I was really under...
465366,E1hLHq6LZ6AoSJzW-oT4FQ,xF2zrVbXtB6TxMBPGJKxog,2.0,2017-07-27 18:21:00,Extremely long wait time. Came here on a Thurs...
465367,O8kmR1np3bcF_--U1vDkYA,5_2Lls-h_shbRGyUruLlTQ,5.0,2015-04-16 02:27:54,Come for the food but stay for the staff! \n\n...


### Data Preprocessing

In [17]:
# Get reviews from "reviews" data file, and combine them
rev = reviews[["business_id","text"]]

In [18]:
rev.head(2)

,business_id,text
4,PMPMHHLe9vFzcR67iCjgLA,Been to this a couple of times. Excellent food...
9,PjtX-5vSTBVyWfMREX8bRA,I'm so sorry to do this to you Capitano and wh...


In [19]:
rev.tail(10)

,business_id,text
465338,JImufwPgbxHdGzoxKE5Ijw,My boyfriend and I stopped by Cafe Bora for de...
465341,7wduNBQUATNkaYzExyGJpg,I am so Happy that Happy Congee is open for bu...
465347,AY4WY_rfrQum3Ad_ohNhVw,Purchased 6 samosas of different fillings with...
465351,_xAJZOKBMPOe47p1MphB2w,We finally went here after seeing many picture...
465360,ICFYlS9nwsoAyaiRXMu9tQ,Fluffy and light pancakes! These reminded me o...
465361,9tfw-OEfpF0qC2hSzRks6g,Location is great. Space is small and tables t...
465363,G2Z-4U6cDsDQvT6v3FzYmA,We were here for dinner and I was really under...
465366,xF2zrVbXtB6TxMBPGJKxog,Extremely long wait time. Came here on a Thurs...
465367,5_2Lls-h_shbRGyUruLlTQ,Come for the food but stay for the staff! \n\n...
465368,9tfw-OEfpF0qC2hSzRks6g,My girlfriend and I were looking for a late ni...


In [20]:
# Add business categories to the comments
part_2 = bus[["business_id","categories"]]
part_2 = part_2.rename(columns={"categories": "text"})
reviews = pd.concat([rev,part_2])

In [21]:
ReviewSummary = reviews.groupby("business_id")["text"].apply(list)
ReviewSummary = ReviewSummary.to_frame()

In [22]:
ReviewSummary.reset_index(inplace=True)

In [23]:
ReviewSummary.head(3)

,business_id,text
0,-0RRiWDtfnS16AKCtfvBZg,[The Independent (without an E?) coffee shop t...
1,-2EAyppKR_2xuLyvJZEbRQ,[I've tried coming here on 2 separate occasion...
2,-2TBP3ZGu7M-FmfoNJvbrQ,[We've been to Rasher's several times and have...


In [ ]:
ReviewSummary.head(1).text.values

array([list(["I used to come to this restaurant about two years ago. I loved them! Fresh food, friendly service, excellent coffee.\n\nI went there today and was not impressed. We waited about five minutes for a table, which we were fine with. They cleaned it quickly and we sat down. However, I noticed ants crawling on the corner of my table. They were babies, so this isn't a serious problem in the restaurant, yet. I couldn't get the attention of the waiter to get the table cleaned, so I killed then with a sugar packet and let it be. \n\nWe ordered coffee, which came quickly. Our cups were dirty, with clear stains on the sides. We waited about ten minutes for refills, which I find is normal. \n\nWe placed our order. The food came quickly, and was okay. I came with my sister and my friend. \n\nMy friend chose to create his own meal, which came on separate plates. \n\nMy sisters eggs were over cooked. \n\nMy food was okay. It was not very presentable, but was delicious. \n\nAll in all, it

In [24]:
ReviewSummary["text"] = ReviewSummary["text"].apply(lambda x: [str(item) for item in x])

In [25]:
len(ReviewSummary["business_id"].unique())

2946

In [26]:
ReviewSummary.head(3)

,business_id,text
0,-0RRiWDtfnS16AKCtfvBZg,[The Independent (without an E?) coffee shop t...
1,-2EAyppKR_2xuLyvJZEbRQ,[I've tried coming here on 2 separate occasion...
2,-2TBP3ZGu7M-FmfoNJvbrQ,[We've been to Rasher's several times and have...


In [27]:
# tokenize data
regEx = re.compile('[^a-z]+')
def tokenize_reviews(text):
  text = text.lower()
  text = regEx.sub(" ", text).strip()
  return text

In [28]:
# function for tokenizing summary
regEx = re.compile("[^a-z]+")
def cleanReviews(reviewText):
  for i in range(len(reviewText)):
    reviewText[i] = reviewText[i].lower()
    reviewText[i] = regEx.sub(" ", reviewText[i]).strip()
  return reviewText

In [29]:
# Tokenize word
ReviewSummary["summaryClean"] = ReviewSummary["text"].apply(cleanReviews)
ReviewSummary.head(3)

,business_id,text,summaryClean
0,-0RRiWDtfnS16AKCtfvBZg,[the independent without an e coffee shop took...,[the independent without an e coffee shop took...
1,-2EAyppKR_2xuLyvJZEbRQ,[i ve tried coming here on separate occasions ...,[i ve tried coming here on separate occasions ...
2,-2TBP3ZGu7M-FmfoNJvbrQ,[we ve been to rasher s several times and have...,[we ve been to rasher s several times and have...


In [30]:
ReviewSummary["review"] = ReviewSummary["summaryClean"].apply(lambda x: " ".join(x))
ReviewSummary.head(3)

,business_id,text,summaryClean,review
0,-0RRiWDtfnS16AKCtfvBZg,[the independent without an e coffee shop took...,[the independent without an e coffee shop took...,the independent without an e coffee shop took ...
1,-2EAyppKR_2xuLyvJZEbRQ,[i ve tried coming here on separate occasions ...,[i ve tried coming here on separate occasions ...,i ve tried coming here on separate occasions o...
2,-2TBP3ZGu7M-FmfoNJvbrQ,[we ve been to rasher s several times and have...,[we ve been to rasher s several times and have...,we ve been to rasher s several times and have ...


In [31]:
# remove stop words
stop_words = set(stopwords.words("english"))
stemmer = SnowballStemmer("english")

def remove_stopwords_stem(text):
  word_tokens = word_tokenize(str(text))
  filtered_sentence = list()
  for w in word_tokens:
    w = regEx.sub(" ", w).strip()
    if w not in stop_words and len(w)>0:
      filtered_sentence.append(stemmer.stem(w))  
  return filtered_sentence

In [32]:
ReviewSummary["text_sw_removed"] = ReviewSummary["review"].apply(remove_stopwords_stem)
ReviewSummary.head(3)  

,business_id,text,summaryClean,review,text_sw_removed
0,-0RRiWDtfnS16AKCtfvBZg,[the independent without an e coffee shop took...,[the independent without an e coffee shop took...,the independent without an e coffee shop took ...,"[independ, without, e, coffe, shop, took, old,..."
1,-2EAyppKR_2xuLyvJZEbRQ,[i ve tried coming here on separate occasions ...,[i ve tried coming here on separate occasions ...,i ve tried coming here on separate occasions o...,"[tri, come, separ, occas, weekday, work, aroun..."
2,-2TBP3ZGu7M-FmfoNJvbrQ,[we ve been to rasher s several times and have...,[we ve been to rasher s several times and have...,we ve been to rasher s several times and have ...,"[rasher, sever, time, realli, impress, consist..."


### Fit & Train Word2Vec

In [33]:
model = Word2Vec(size = 100, min_count = 5,sg=1,window=5)

In [34]:
sentences = [row for row in ReviewSummary["text_sw_removed"]]

In [35]:
len(sentences)

2946

In [36]:
model.build_vocab(sentences)

In [37]:
model.train(sentences,total_examples = model.corpus_count,epochs = model.iter)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `iter` (Attribute will be removed in 4.0.0, use self.epochs instead).
  """Entry point for launching an IPython kernel.


(26445604, 34808035)

In [38]:
model.most_similar("chines",topn=10)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('cantones', 0.7513747811317444),
 ('asian', 0.7215375900268555),
 ('hakka', 0.6770930290222168),
 ('yunnan', 0.6744613647460938),
 ('shanghaines', 0.6666659712791443),
 ('trad', 0.6636523008346558),
 ('dimsum', 0.6613206267356873),
 ('rame', 0.6519206762313843),
 ('sum', 0.6497119069099426),
 ('china', 0.6477807760238647)]

In [39]:
model.most_similar("taco",topn=10)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('carnita', 0.8330883383750916),
 ('baja', 0.8186402320861816),
 ('asada', 0.7899429798126221),
 ('taquito', 0.7813836932182312),
 ('pastor', 0.7795311808586121),
 ('tinga', 0.765974760055542),
 ('quesadilla', 0.7472147345542908),
 ('barbacoa', 0.7466354370117188),
 ('flauta', 0.741124153137207),
 ('chimichanga', 0.7351001501083374)]

In [40]:
model.most_similar("italian",topn=10)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('napoli', 0.7017148733139038),
 ('neapolitan', 0.6895362138748169),
 ('nonna', 0.6626576781272888),
 ('roman', 0.6599322557449341),
 ('romagna', 0.6585886478424072),
 ('terroni', 0.6448279619216919),
 ('itali', 0.6406687498092651),
 ('neopolitan', 0.6366987228393555),
 ('falasca', 0.6353523135185242),
 ('austrian', 0.6351158022880554)]

In [41]:
model.wv.save_word2vec_format('word2vec_ver4.vector')

In [42]:
# get the sentence vector using average value of word vector.
# if a word not in vocabulary, see it as 0
def sentence_vector(sentence):
  vecs = list()
  for word in sentence:
    try:
      vec = model.wv[word]
    except:
      vec = [0] * 100
    vecs.append(vec)
  vector = np.mean(vecs, axis=0)
  return vector

In [43]:
ReviewSummary["vector"] = ReviewSummary["text_sw_removed"].apply(sentence_vector)

In [44]:
ReviewSummary.head(3)

,business_id,text,summaryClean,review,text_sw_removed,vector
0,-0RRiWDtfnS16AKCtfvBZg,[the independent without an e coffee shop took...,[the independent without an e coffee shop took...,the independent without an e coffee shop took ...,"[independ, without, e, coffe, shop, took, old,...","[0.20588197, 0.12743695, 0.1628587, -0.1272542..."
1,-2EAyppKR_2xuLyvJZEbRQ,[i ve tried coming here on separate occasions ...,[i ve tried coming here on separate occasions ...,i ve tried coming here on separate occasions o...,"[tri, come, separ, occas, weekday, work, aroun...","[0.19634292, 0.069451794, 0.20247477, -0.05517..."
2,-2TBP3ZGu7M-FmfoNJvbrQ,[we ve been to rasher s several times and have...,[we ve been to rasher s several times and have...,we ve been to rasher s several times and have ...,"[rasher, sever, time, realli, impress, consist...","[0.1866759128601333, 0.10649319170656199, 0.21..."


In [45]:
len(list(ReviewSummary.vector)[4])

100

In [46]:
def cos_similarity(v1,v2):
  return np.dot(v1, v2) / np.sqrt(np.dot(v1, v1)) / np.sqrt(np.dot(v2, v2))

In [47]:
ReviewSummary.shape

(2946, 6)

In [48]:
# Save this csv file
ReviewSummary.to_csv('input_data_ver4.csv', index=False)

### Keyword Recommend(SimHash, Loc)

In [ ]:
!pip install annoy

4/1AY0e-g69UGZPi2jDViECHDTlG3gLaQ4cYGlOuqAePo633jpdoi7wDuLB4AcCollecting annoy
     |████████████████████████████████| 655kB 5.4MB/s 
  Created wheel for annoy: filename=annoy-1.17.0-cp36-cp36m-linux_x86_64.whl size=390360 sha256=08c8bcd3ad47eefe642f670bd292e07fe5f028853e9dce86ac010404ccceefda
  Stored in directory: /root/.cache/pip/wheels/3a/c5/59/cce7e67b52c8e987389e53f917b6bb2a9d904a03246fadcb1e
Successfully built annoy


In [ ]:
# use simhash to find the most similar items
from annoy import AnnoyIndex

In [ ]:
ReviewSummary.head(3)

,business_id,text,review,summaryClean,text_sw_removed,vector
0,--DaPTJW3-tB1vP-PfdTEg,[i used to come to this restaurant about two y...,i used to come to this restaurant about two ye...,[i used to come to this restaurant about two y...,"[use, come, restaur, two, year, ago, love, fre...","[-0.14509956251138562, 0.08304661644430762, -0..."
1,--SrzpvFLwP_YFwB_Cetow,[mochipopcorn rating chart averaged star ratin...,mochipopcorn rating chart averaged star rating...,[mochipopcorn rating chart averaged star ratin...,"[mochipopcorn, rate, chart, averag, star, rate...","[-0.2001813211278641, 0.17022376294073224, -0...."
2,--Y1Adl1YUWfYIRSd8vkmA,[this eaton centre geox store uses fraudulent ...,this eaton centre geox store uses fraudulent s...,[this eaton centre geox store uses fraudulent ...,"[eaton, centr, geox, store, use, fraudul, sale...","[-0.028528770014736953, 0.21434751628169876, 0..."


In [ ]:
def keyword_recommend_loc_simhash(input_str, df_old, lat,log):
  lat_min = lat-0.07/2
  lat_max = lat+0.07/2
  log_min = log-0.08/2
  log_max = log+0.08/2 
  bus_id = set(bus[(bus.latitude>=lat_min)&(bus.latitude<=lat_max)&(bus.longitude<=log_max)&(bus.longitude>=log_min)].business_id)
  text_token = tokenize_reviews(input_str)
  text_token_sw_removed = remove_stopwords_stem(text_token)
  vector = sentence_vector(text_token_sw_removed)
  df = df_old[df_old["business_id"].isin(bus_id)]

  # build simhash
  res_vec = list(df.vector)
  f = 100 # dim
  t = AnnoyIndex(f) # length of item vector that will be indexed
  for i in range(len(res_vec)):
    t.add_item(i,res_vec[i])
  t.build(30) # 30 trees

  # Get top 10 recommendations
  indexes = t.get_nns_by_vector(vector, 10)
  result = pd.DataFrame(columns=["business_id"])
  for index in indexes:
    result = result.append({"business_id": df.iloc[index].business_id},ignore_index=True)
  return result

In [ ]:
def top_10_loc_hash(kword, df10,lat,log):
  recommend_list = keyword_recommend_loc_simhash(kword, df10,lat,log)
  return recommend_list

In [ ]:
# Get the business id information
table = bus[["business_id", "name", "categories", "rating"]]
table.head(3)

,business_id,name,categories,rating
0,EosRKXIGeSWFYWwpkbhNnA,Xtreme Couture,"Martial Arts, Gyms, Fitness & Instruction, Act...",3.0
1,1wWneWD_E1pBIyVpdHMaQg,Air Jordan Store,"Shopping, Shoe Stores, Fashion",4.0
2,007Dg4ESDVacWcC4Vq704Q,Front Door Organics,"Shopping, Food, Organic Stores, Specialty Food...",4.0


In [ ]:
restaurents = top_10_loc_hash("chinese food", ReviewSummary, 43.66346, -79.38746)
restaurents

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:15: FutureWarning: The default argument for metric will be removed in future version of Annoy. Please pass metric='angular' explicitly.
  from ipykernel import kernelapp as app


,business_id
0,qoDrPDPEGRsF8I_VgYftjQ
1,DEdB-wkPV1QzcSFMTTYl-w
2,9pBcKw24z9nZiHW7Vx_z8A
3,TMUPcmHwUtK3UV-9Rfs_ew
4,fuk8V-SmFO0xRyC9eMDarw
5,OaO_5YeBF9xBXmcgG1t0WA
6,S-BgFH8oqD5n4OsDNhxr9g
7,Koofpyww8Y-H9O7vw8mapQ
8,95A_eit7YDcEtM7zPO9yHQ
9,n-ps87_nlV56dKLD7Wep3w


In [ ]:
# Merge table
recommendation = restaurents.merge(table, on="business_id")
recommendation

,business_id,name,categories,rating
0,qoDrPDPEGRsF8I_VgYftjQ,Wikki Hut,"Fast Food, Food Court, Restaurants, Salad",3.5
1,DEdB-wkPV1QzcSFMTTYl-w,South Sea Chinese Foods,"Ethnic Food, Food, Chinese, Restaurants, Speci...",3.0
2,9pBcKw24z9nZiHW7Vx_z8A,Manchu Wok,"Restaurants, Fast Food, Chinese, Asian Fusion",3.5
3,TMUPcmHwUtK3UV-9Rfs_ew,Wokking On Wheels,"Food Trucks, Food",3.5
4,fuk8V-SmFO0xRyC9eMDarw,China Garden Restaurant,"Indian, Chinese, Food, Specialty Food, Restaur...",2.5
5,OaO_5YeBF9xBXmcgG1t0WA,Indochine Thai Vietnamese Restaurant,"Vietnamese, Thai, Food, Restaurants, Specialty...",3.0
6,S-BgFH8oqD5n4OsDNhxr9g,Joyful House Restaurant,"Chinese, Restaurants",3.0
7,Koofpyww8Y-H9O7vw8mapQ,Shanghai 360,"Dim Sum, Chinese, Restaurants",3.0
8,95A_eit7YDcEtM7zPO9yHQ,Bamboo Cafe Chinese Restaurant,"Chinese, Restaurants",4.0
9,n-ps87_nlV56dKLD7Wep3w,Fookee Express,"Chinese, Fast Food, Restaurants",3.0


### Mexiacan Food

In [ ]:
restaurents = top_10_loc_hash("mexican food", ReviewSummary, 43.66346, -79.38746)
recommendation = restaurents.merge(table, on="business_id")
recommendation

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:15: FutureWarning: The default argument for metric will be removed in future version of Annoy. Please pass metric='angular' explicitly.
  from ipykernel import kernelapp as app


,business_id,name,categories,rating
0,Q1jLolGK7XvP4QNn6Hl0_w,Willow Restaurant,"Restaurants, Tex-Mex",3.0
1,iCtMVLywXMzCQAfMKRSUyg,The Mexican Burrito,"Restaurants, Mexican",4.0
2,EtG-mGU0rYzxPlYY_nCQfg,El Trompo Movil,"Food, Food Trucks",4.0
3,DaTQLers4x_DbGPi7r_xVw,Rebozos College,"Restaurants, Mexican",3.5
4,17HGaSwTNL-9JKWi8oAgqA,Taco Bell,"Mexican, American (Traditional), Restaurants, ...",3.0
5,1W_gZM_uuEvJqapbIl6z9Q,El Charro,"Food Stands, Food, Mexican, Street Vendors, Re...",4.0
6,Bf2fuqWbHd3L-X69FSMvmg,El Trompo Taco Bar,"Restaurants, Mexican",4.0
7,YiFCHXw7oZQg53oM3e0R4g,Mucho Burrito,"Fast Food, Mexican, Tex-Mex, Restaurants",2.5
8,NkcT0GT7Xt2mtvIgHSxOcw,La Burrita,"Mexican, Restaurants",3.5
9,Io1WmI1cUc8QaUVxSx1KuA,Perola Supermarket,"Restaurants, Food, Grocery, Latin American",4.0


### Italian Food

In [ ]:
restaurents = top_10_loc_hash("italian food", ReviewSummary, 43.66346, -79.38746)
recommendation = restaurents.merge(table, on="business_id")
recommendation

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:15: FutureWarning: The default argument for metric will be removed in future version of Annoy. Please pass metric='angular' explicitly.
  from ipykernel import kernelapp as app


,business_id,name,categories,rating
0,R2BKGMHjxSDWyFVJ44tONQ,RIstorante Marzitelli,"Italian, French, Breakfast & Brunch, Restaurants",5.0
1,ausSJnSyp7En_bmZQCGc9Q,Panini Trip,"Restaurants, French, Italian",2.5
2,0rMFhLzeq3Z2OaJ62MQXlw,Spiga Ristorante,"Restaurants, Italian",4.0
3,MPhhbu9HfDK0OjdtdFN1BA,Pizza Pizza,"Food, Pizza, Italian, Restaurants, Chicken Wings",2.0
4,-smUKwiwkWmz1qWUcgqYsg,Cafe Oro Di Napoli,"Italian, Restaurants, Pizza",4.5
5,PhdbEJEHGLtKs-nSFM3Kxg,Black Skirt,"Restaurants, Italian",2.5
6,DvXpWBz5_-hZQuvu-RnSwQ,The Food Shop,"Italian, Specialty Food, Food, Restaurants",3.5
7,o954oK-S2OxH-OaBc7f0UQ,Rosina,"Pizza, Italian, Restaurants",3.5
8,bo4loBeLwkQqZ7HGrgxXRQ,Trattoria Taverniti,"Pizza, Italian, Restaurants, Beer, Wine & Spir...",4.0
9,jbbFyy2rl8x_7ssz-GifbQ,Fusilli Restaurant,"Italian, Pizza, Restaurants",3.5


In [ ]:
restaurents = top_10_loc_hash("seafood lobster", ReviewSummary, 43.66346, -79.38746)
recommendation = restaurents.merge(table, on="business_id")
recommendation

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:15: FutureWarning: The default argument for metric will be removed in future version of Annoy. Please pass metric='angular' explicitly.
  from ipykernel import kernelapp as app


,business_id,name,categories,rating
0,doK6H6hraPaVM6X5QE6d7Q,Bluenose Lobster,"Seafood, Restaurants, Shopping, Seafood Market...",3.0
1,Dvhg57y-SOcKty6WTG1XDw,Boil King,"Chicken Wings, Bars, Cajun/Creole, Nightlife, ...",3.5
2,t-UBWPiyUgXNC7T9-haVHA,Seafront Fish Market,"Specialty Food, Food, Seafood Markets",4.0
3,KIs5vcQLVExjxtEOBNkXdA,The Boil Bar,"Restaurants, Cajun/Creole, Seafood",3.5
4,agWZA6ajLvu6MfFySNBzZw,The Captain's Boil,"Seafood, Asian Fusion, Restaurants, Cajun/Creole",3.5
5,bLtC8xJXkd1b6MT0KjnSvw,The Captain's Boil,"Restaurants, Seafood, Asian Fusion, Cajun/Creole",3.5
6,wZhB4U003eg2GQ-F7D2D4A,The Captain's Boil,"Seafood, Asian Fusion, Restaurants, Cajun/Creole",3.5
7,N6tJKe0h1NJgVm86NDg7ug,lbs. Restaurant,"Salad, Seafood, Steakhouses, Burgers, Canadian...",3.5
8,xL-Ckvr8k8vEBJxC1wsCqQ,Wah Sing Seafood Restaurant,"Chinese, Seafood, Restaurants",3.0
9,EWmwbOm_4UhOtvLaBzHpPA,Fresh Off The Boat,"Fish & Chips, Specialty Food, Seafood, Food, S...",3.5


### Doc2Vec Model

In [ ]:
import gensim

In [ ]:
# tokenize data
regEx = re.compile('[^a-z]+')
def tokenize_reviews(text):
  text = text.lower()
  text = regEx.sub(" ", text).strip()
  return text

In [ ]:
# remove stop words
stop_words = set(stopwords.words("english"))

def remove_stopwords_stem(text):
  stemmer = SnowballStemmer("english")
  word_tokens = word_tokenize(text)
  filtered_sentence = [stemmer.stem(w) for w in word_tokens if w not in stop_words]
  return filtered_sentence

In [ ]:
ReviewSummary.head(3)

,business_id,text,review,summaryClean,text_sw_removed
0,--DaPTJW3-tB1vP-PfdTEg,['i used to come to this restaurant about two ...,i used to come to this restaurant about two ye...,['i used to come to this restaurant about two ...,"['use', 'come', 'restaur', 'two', 'year', 'ago..."
1,--SrzpvFLwP_YFwB_Cetow,['mochipopcorn rating chart averaged star rati...,mochipopcorn rating chart averaged star rating...,['mochipopcorn rating chart averaged star rati...,"['mochipopcorn', 'rate', 'chart', 'averag', 's..."
2,--Y1Adl1YUWfYIRSd8vkmA,['this eaton centre geox store uses fraudulent...,this eaton centre geox store uses fraudulent s...,['this eaton centre geox store uses fraudulent...,"['eaton', 'centr', 'geox', 'store', 'use', 'fr..."


In [ ]:
ReviewSummary.head(3).text_sw_removed.values

array(["['use', 'come', 'restaur', 'two', 'year', 'ago', 'love', 'fresh', 'food', 'friend', 'servic', 'excel', 'coffe', 'went', 'today', 'impress', 'wait', 'five', 'minut', 'tabl', 'fine', 'clean', 'quick', 'sat', 'howev', 'notic', 'ant', 'crawl', 'corner', 'tabl', 'babi', 'serious', 'problem', 'restaur', 'yet', 'get', 'attent', 'waiter', 'get', 'tabl', 'clean', 'kill', 'sugar', 'packet', 'let', 'order', 'coffe', 'came', 'quick', 'cup', 'dirti', 'clear', 'stain', 'side', 'wait', 'ten', 'minut', 'refil', 'find', 'normal', 'place', 'order', 'food', 'came', 'quick', 'okay', 'came', 'sister', 'friend', 'friend', 'chose', 'creat', 'meal', 'came', 'separ', 'plate', 'sister', 'egg', 'cook', 'food', 'okay', 'present', 'delici', 'okay', 'experi', 'better', 'restaur', 'choic', 'would', 'go', 'breakfast', 'place', 'st', 'clair', 'realli', 'enjoy', 'breakfast', 'restaur', 'west', 'end', 'citi', 'came', 'time', 'brought', 'immedi', 'famili', 'enjoy', 'breakfast', 'thorough', 'servic', 'warm', 'frie

In [ ]:
ReviewSummary.dtypes

business_id        object
text               object
review             object
summaryClean       object
text_sw_removed    object
dtype: object

In [ ]:
def tagged_document(list_of_list_of_words):
   for i, list_of_words in enumerate(list_of_list_of_words):
      yield gensim.models.doc2vec.TaggedDocument(list_of_words, [i])

In [ ]:
ls_replies = [i for i in ReviewSummary["text_sw_removed"]]

In [ ]:
# convert the type of ls_replies
for i in range(len(ls_replies)):
  temp = ls_replies[i].strip('][').split(', ')
  for j in range(len(temp)):
    temp[j] = regEx.sub(" ", temp[j]).strip()
  ls_replies[i] = temp  

In [ ]:
ls_replies[2][0]

'eaton'

In [ ]:
data_for_training = list(tagged_document(ls_replies))

In [ ]:
data_for_training[:5]

[TaggedDocument(words=['use', 'come', 'restaur', 'two', 'year', 'ago', 'love', 'fresh', 'food', 'friend', 'servic', 'excel', 'coffe', 'went', 'today', 'impress', 'wait', 'five', 'minut', 'tabl', 'fine', 'clean', 'quick', 'sat', 'howev', 'notic', 'ant', 'crawl', 'corner', 'tabl', 'babi', 'serious', 'problem', 'restaur', 'yet', 'get', 'attent', 'waiter', 'get', 'tabl', 'clean', 'kill', 'sugar', 'packet', 'let', 'order', 'coffe', 'came', 'quick', 'cup', 'dirti', 'clear', 'stain', 'side', 'wait', 'ten', 'minut', 'refil', 'find', 'normal', 'place', 'order', 'food', 'came', 'quick', 'okay', 'came', 'sister', 'friend', 'friend', 'chose', 'creat', 'meal', 'came', 'separ', 'plate', 'sister', 'egg', 'cook', 'food', 'okay', 'present', 'delici', 'okay', 'experi', 'better', 'restaur', 'choic', 'would', 'go', 'breakfast', 'place', 'st', 'clair', 'realli', 'enjoy', 'breakfast', 'restaur', 'west', 'end', 'citi', 'came', 'time', 'brought', 'immedi', 'famili', 'enjoy', 'breakfast', 'thorough', 'servic',

In [ ]:
model = gensim.models.doc2vec.Doc2Vec(vector_size=100, min_count=5, epochs=10,window=5)

In [ ]:
model.build_vocab(data_for_training)

In [ ]:
model.train(data_for_training, total_examples=model.corpus_count, epochs=model.epochs)

In [ ]:
model.save("doc2vec_ver3.vec")

In [ ]:
model.docvecs[0]

array([-1.2794054 , -0.4880025 ,  0.14548911, -0.05196824,  2.0980856 ,
       -0.72438496,  0.23473446,  0.26780754,  1.0918875 ,  0.6872864 ,
        0.05933126, -0.7950314 ,  0.48285708, -0.2089741 ,  0.43964437,
       -1.3041108 ,  0.64572996,  1.4042834 ,  0.01081742,  0.09910483,
        1.4079615 , -0.5996914 ,  0.71393055,  0.62786573,  1.6291684 ,
        0.6962231 , -1.5517378 ,  0.9287335 , -1.2051002 , -0.22022203,
        0.41097438,  0.2655464 , -0.44164595,  0.6217793 ,  0.05661397,
        0.3134297 , -1.3999383 ,  0.36375087, -0.43588305,  1.6319321 ,
       -0.32012308,  1.8272043 , -0.86594504, -0.4583026 ,  0.676646  ,
       -0.7540901 , -0.22287251,  0.7747064 , -0.6768775 ,  0.2997839 ,
       -0.37968424, -1.1189358 ,  0.7837753 , -0.22112684, -0.09497897,
        1.4596932 ,  0.20325196, -1.795444  ,  0.74730945,  0.87049884,
       -0.3367143 , -0.77564293,  0.84305006,  2.1604066 ,  2.5637746 ,
        1.2928748 ,  0.22097929,  0.43453804, -0.5211395 ,  0.14

In [ ]:
def keyword_recommend_loc(input_str, df_old, lat,log):
  lat_min = lat-0.07/2
  lat_max = lat+0.07/2
  log_min = log-0.08/2
  log_max = log+0.08/2 
  bus_id = set(bus[(bus.latitude>=lat_min)&(bus.latitude<=lat_max)&(bus.longitude<=log_max)&(bus.longitude>=log_min)].business_id)
  text_token = tokenize_reviews(input_str)
  text_token_sw_removed = remove_stopwords_stem(text_token)
  #print(text_token_sw_removed)
  infer_vector = model.infer_vector(text_token_sw_removed)
  #print(infer_vector)
  #df = df_old[df_old["business_id"].isin(bus_id)]
  #ls_replies = [i for i in df["text_sw_removed"]]

  # Get top 10 recommendations
  #indexes = t.get_nns_by_vector(vector, 10)
  similar_docs = model.docvecs.most_similar([infer_vector], topn = 50)
  result = pd.DataFrame(columns=["business_id"])
  count = 0
  for tup in similar_docs:
    #print(tup)
    if df_old.iloc[tup[0]]["business_id"] in (bus_id) and count < 10:
      result = result.append({"business_id": df_old.iloc[tup[0]].business_id},ignore_index=True)
      count += 1
  return result

In [ ]:
def top_10_loc(kword, df10,lat,log):
  recommend_list = keyword_recommend_loc(kword, df10,lat,log)
  return recommend_list

In [ ]:
restaurents = top_10_loc("chinese", ReviewSummary, 43.66346, -79.38746)
recommendation = restaurents.merge(table, on="business_id")
recommendation

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


,business_id,name,categories,rating
0,n-ps87_nlV56dKLD7Wep3w,Fookee Express,"Chinese, Fast Food, Restaurants",3.0
1,qoDrPDPEGRsF8I_VgYftjQ,Wikki Hut,"Fast Food, Food Court, Restaurants, Salad",3.5
2,BTdZW9U4LwDkqJMMV8vPCA,Wuji Xuan Life Wellness,"Massage Therapy, Active Life, Reiki, Qi Gong, ...",5.0
3,V7tMelCxBIox5syA3Tw1Tg,BODE Spa for Men,"Barbers, Beauty & Spas, Massage, Hair Removal,...",3.0
4,Dwb8cSY4iTecH0ZC7qrElg,Pearl Alteration,"Local Services, Sewing & Alterations",3.5
5,psTYC2HcSIC7uUX_3kEKiA,Airport Limo Canada,"Airport Shuttles, Tours, Taxis, Limos, Hotels ...",5.0
6,kpwEHFQZ3wK1uWwJB1Ri9g,Airport Limo Toronto,"Transportation, Hotels & Travel, Taxis, Limos,...",1.0
7,O_V5WyNTupAKmunlpBCShw,Toronto Airport Limo Taxi,"Hotels & Travel, Limos, Taxis, Airport Shuttle...",2.5
8,iBH1TeUrJn1VuL59J1StUg,Eastern legend,"Chinese, Restaurants",3.0
9,8GOb-i1qrOskOtcca_BJUA,Blue Ocean Moving,"Home Services, Movers",1.0


In [ ]:
# Merge table
recommendation = restaurents.merge(table, on="business_id")
recommendation

,business_id,name,categories,rating
0,tSekZuxt-9UsBXJzTjGjog,Porcelain Room,"Eyelash Service, Skin Care, Hair Removal, Beau...",5.0
1,V7tMelCxBIox5syA3Tw1Tg,BODE Spa for Men,"Barbers, Beauty & Spas, Massage, Hair Removal,...",3.0
2,Dwb8cSY4iTecH0ZC7qrElg,Pearl Alteration,"Local Services, Sewing & Alterations",3.5
3,8GOb-i1qrOskOtcca_BJUA,Blue Ocean Moving,"Home Services, Movers",1.0
4,qzurbSMR1it_Rr0lusGmug,"Lawrence Tong, MD, FACS, FRCSC","Beauty & Spas, Medical Spas, Surgeons, Cosmeti...",4.0
5,NLnci3vWOqGAHA_bdwQdww,Welcare,"Health & Medical, Family Practice, Doctors",2.5
6,psTYC2HcSIC7uUX_3kEKiA,Airport Limo Canada,"Airport Shuttles, Tours, Taxis, Limos, Hotels ...",5.0
7,iab-dDNMEeEABDReZpRu5g,L'Attitudes Salon & Spa,"Nail Salons, Beauty & Spas, Day Spas, Hair Salons",1.0
8,Xhz55K2W1sBFa1MRiDuWfQ,Hotel 89,"Hostels, Hotels & Travel, Hotels, Event Planni...",1.0
9,mPwWuN17o3elnJrNJJf-3Q,Rosen Avra Barrister & Family Law Mediator,"Lawyers, Professional Services, Divorce & Fami...",1.0


In [ ]:
model.wv.save_word2vec_format('doc2vec_ver3.vector')

### Fried Chicken

In [ ]:
# University of Toronto - St. George Campus:
restaurents = top_10_loc("Fried Chiecken", ReviewSummary, 43.66311072883239, -79.39440813687946)
recommendation = restaurents.merge(table, on="business_id")
recommendation

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


,business_id,name,categories,rating
0,wRvIH4SErkysd9JondMqeg,Urban Vapour,"Shopping, Tobacco Shops, Vape Shops",1.0
1,QnpXjPjMdGAhXwRz1uR5eQ,Cherry Paul M H Dr Eye Phys & Surg,"Health & Medical, Doctors",1.0
2,v2M1CdkjnF0yT0fkRIc-qg,Bilingual Source-Source Bilingue,"Professional Services, Employment Agencies",3.5
3,6gk_JaGhYq2RoP9Wa7xSfA,"Josephson Robert, MD","Health & Medical, Doctors",2.5
4,MKUavfc1tE2zw1Sw7H3mOw,"Trevor Vander Doelen, BSC, DC","Chiropractors, Physical Therapy, Health & Medi...",5.0
5,Xhz55K2W1sBFa1MRiDuWfQ,Hotel 89,"Hostels, Hotels & Travel, Hotels, Event Planni...",1.0
6,idJC7VSGOWCD0ZLUOPxuFQ,Acsam Clinic,"Beauty & Spas, Massage, Acupuncture, Health & ...",4.5
7,QW0ozABysFmiOI7oVFrbfQ,Forest Hill Orthodontics,"Orthodontists, Dentists, Health & Medical",3.5
8,7KZA-a0kfhv-F7IuHr-JCA,Bay St. Eyecare,"Health & Medical, Optometrists, Shopping, Eyew...",5.0
9,C8enIVsTlavMkViGnWoweg,Turkish Airlines,"Airlines, Hotels & Travel, Transportation",2.0


In [ ]:
# Scarborough Civic Centre
restaurents = top_10_loc("Fried Chiecken", ReviewSummary, 43.77952047125285, -79.25780882879529)
recommendation = restaurents.merge(table, on="business_id")
recommendation

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


,business_id,name,categories,rating
0,ayQMJlNXdOsXhOT4hhSj6g,a-32 Wireless,"Local Services, Mobile Phone Repair, Electroni...",2.5


In [ ]:
# Aga Khan Museum
restaurents = top_10_loc("Friend Chicken", ReviewSummary, 43.72803030610192, -79.32977026321979)
recommendation = restaurents.merge(table, on="business_id")
recommendation

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


,business_id,name,categories,rating
0,pR-2JcFQfm9g7hO-BShFsw,Greek Gyro Grill,"Greek, Mediterranean, Restaurants",3.0
1,JkEXRFeYyMYaP4ZYZ-BWWQ,Belyea Brothers Heating and Cooling,"Contractors, Home Services, Plumbing, Heating ...",3.5
2,ze5kafJm7b-BP6p7tsN3-g,Danilo Martinez - King Realty Group,"Home Services, Real Estate Agents, Real Estate...",5.0
3,QrfBSMFmAw8HeOXESEwXhQ,Nail Princess,"Beauty & Spas, Nail Salons",1.5
4,Gbz3fzulgXZ84nySj_TgKA,"Shawn Meirovici, ND","Health & Medical, Naturopathic/Holistic, Doctors",5.0
5,Xg5_jD0UlErBdERYh1K7rw,Speaking of Dogs Rescue,"Pet Adoption, Pet Services, Pets",2.5


### BM25

In [49]:
!pip install rank_bm25 nltk

In [50]:
from rank_bm25 import BM25Okapi

In [ ]:
sentences = [row for row in ReviewSummary["text_sw_removed"]]

In [51]:
bm25 = BM25Okapi(sentences)

In [55]:
# Get the business id information
table = bus[["business_id", "name", "categories", "rating"]]
table.head(3)

,business_id,name,categories,rating
3,rVBPQdeayMYht4Uv_FOLHg,Gourmet Burger Company,"Restaurants, Burgers, Food",3.0
11,8k62wYhDVq1-652YbJi5eg,Tim Hortons,"Bagels, Donuts, Food, Cafes, Coffee & Tea, Res...",3.0
14,0DnQh8SE8BSnvJltGCCiWg,Chick-N-Joy,"Fast Food, Restaurants, Chicken Shop",3.0


In [53]:
def BM25_recommend(input_str, df_old, lat,log):
  lat_min = lat-0.07/2
  lat_max = lat+0.07/2
  log_min = log-0.08/2
  log_max = log+0.08/2 
  bus_id = set(bus[(bus.latitude>=lat_min)&(bus.latitude<=lat_max)&(bus.longitude<=log_max)&(bus.longitude>=log_min)].business_id)
  text_token = tokenize_reviews(input_str)
  text_token_sw_removed = remove_stopwords_stem(text_token)
  vector = sentence_vector(text_token_sw_removed)
  df = df_old[df_old["business_id"].isin(bus_id)]
  sentences = [row for row in df["text_sw_removed"]]
  bm25 = BM25Okapi(sentences)
  docs = bm25.get_top_n(text_token_sw_removed, df.text_sw_removed.values, n=10)
  result = pd.DataFrame(columns=["business_id"])
  for i in range(len(df)):
    if df.iloc[i].text_sw_removed in docs:
      result = result.append({"business_id": df.iloc[i].business_id},ignore_index=True)
  recommendation = result.merge(table, on="business_id")
  return recommendation

### Fried Chicken

In [58]:
# University of Toronto - St. George Campus:
recommendation = BM25_recommend("Fried Chiecken", ReviewSummary, 43.66311072883239, -79.39440813687946)
recommendation

,business_id,name,categories,rating
0,6ueacQ4py4qK5Gp9BFGvnw,Mr Tasty Fries,"Restaurants, Food Trucks, Food Stands, Fast Fo...",4.0
1,D4ETxxeNQEnSw_e1H46DXQ,Monga Fried Chicken,"Restaurants, Taiwanese, Fast Food, Asian Fusion",3.5
2,EWmwbOm_4UhOtvLaBzHpPA,Fresh Off The Boat,"Fish & Chips, Specialty Food, Seafood, Food, S...",3.5
3,JrB6On8q8f1_ICQSCLg47w,Five Guys,"Restaurants, Burgers, Fast Food, American (Tra...",3.5
4,N93EYZy9R0sdlEvubu94ig,Banh Mi Boys,"Sandwiches, Restaurants, Food, Vietnamese, Asi...",4.5
5,UN0UwUh7jaeX6Jg3lZImCg,Holy Chuck,"Food, Poutineries, Restaurants, Desserts, Burgers",3.5
6,aRqxT28Tdz2aDiZ7Vkyejw,White Brick Kitchen,"Comfort Food, Restaurants, Breakfast & Brunch",4.0
7,e6UDJBEIoXBbnXtdwkqIGA,Hot-Star,"Fast Food, Food, Taiwanese, Chicken Wings, Res...",3.5
8,mQuDZwxxPe54MY4TT8lEPQ,Bar Fancy,"Nightlife, Restaurants, Bars, Chicken Shop, Co...",4.0
9,qKUbPdzNQpdSP2mhk3tREg,Moo Frites,"Restaurants, Food, Belgian, Fast Food, Special...",4.0


In [59]:
recommendation.to_csv("FC_loc1.csv",index=False)

In [60]:
# Scarborough Civic Centre
recommendation = BM25_recommend("Fried Chiecken", ReviewSummary, 43.77952047125285, -79.25780882879529)
recommendation

,business_id,name,categories,rating
0,AIasveRX0245WeAH0C46Uw,Chick Fiesta,"Chicken Shop, South African, African, Portugue...",4.0
1,Eg76PjESw705exvbt_VYjQ,Wu Jian Dao Shanghai Style Restaurant,"Restaurants, Asian Fusion, Chinese, Food, Bubb...",3.5
2,F9tePBgROEAcd8xZqTZUfg,Jollibee,"Fast Food, Chicken Wings, Filipino, Restaurants",3.5
3,HTZFw4QDKhD8KRbnb5ZuJA,Chicken Kitchen,"Restaurants, Chicken Shop, Fast Food, Chicken ...",4.5
4,JTIbsAs3qUq-0bNIh404ng,City Fried Chicken,"Restaurants, Fast Food, Fish & Chips, Cheesest...",4.0
5,Rx_YmoVKPWvzGuFuvd-57w,Chick N Delish,"Restaurants, Fast Food, Chicken Wings",3.5
6,XZV6pSuJkfsyoT9eJc3dWw,Church's Chicken,"Chicken Shop, Fast Food, Restaurants, Chicken ...",3.0
7,_xAJZOKBMPOe47p1MphB2w,Fishman Lobster Clubhouse Restaurant,"Seafood, Restaurants, Chinese, Live/Raw Food",4.0
8,d3JhScywFHns4gdR7vIqlA,The Wing Spot,"Restaurants, Chicken Wings, American (Traditio...",4.0
9,ycAW6Q5quaCSDX5zwQ3tPg,New York Fries,"Restaurants, Food, Fast Food, Specialty Food, ...",3.5


In [61]:
recommendation.to_csv("FC_loc2.csv",index=False)

In [62]:
# Aga Khan Museum
recommendation = BM25_recommend("Friend Chicken", ReviewSummary, 43.72803030610192, -79.32977026321979)
recommendation

,business_id,name,categories,rating
0,-J6FVdY9pSgAdFmmalO-pQ,Jatujak,"Food, Thai, Food Delivery Services, Restaurants",4.0
1,1jNteKQ2JuF6Sk1SI9X23Q,P F Chang's,"Food, Chinese, Restaurants, Asian Fusion, Vege...",3.0
2,ExoqgA3aAz6bYidI0DSQvw,California Sandwiches,"Restaurants, Food, Sandwiches",4.0
3,GZ12skjXwuMuH3LidRN-mA,Chipotle Mexican Grill,"Restaurants, Spanish, Mexican, Fast Food",3.5
4,LG7-BUvDLoOZCHDjyiBQFQ,Dirty Grill,"Salad, Restaurants, Juice Bars & Smoothies, Fo...",4.0
5,Mksyt1TZQ-7vzCZjeuzfAg,CASA manila Restaurant,"Fast Food, Filipino, Restaurants, Chinese",3.0
6,URvh6DgFyA-d6Z7OVKxczg,Rally Sports Bar + Smokehouse,"Smokehouse, Food, Barbeque, Nightlife, Comfort...",4.0
7,YIez_A3WOt9J2SXN7OMa2Q,Allwyn's Bakery,"Restaurants, Bakeries, Caribbean, Food",4.0
8,bwFxrxHrz9I36awGc-yjjw,TAO Northern Chinese Cuisine,"Dim Sum, Seafood, Chinese, Noodles, Food, Rest...",3.5
9,j4htyBDNBzrog6A_byqdDw,WingBurger,"Comfort Food, Chicken Wings, Food, Fast Food, ...",4.5


In [63]:
recommendation.to_csv("FC_loc3.csv",index=False)

### Sushi

In [64]:
# University of Toronto - St. George Campus:
recommendation = BM25_recommend("Sushi", ReviewSummary, 43.66311072883239, -79.39440813687946)
recommendation

,business_id,name,categories,rating
0,5C58ejb2nmJojUFPlvJj2A,Masa Sushi & Teppanyaki,"Restaurants, Japanese, Sushi Bars, Food, Food ...",2.5
1,KRWa12MkvE7sgaEnNhSWtQ,Sushi 930,"Sandwiches, Restaurants, Cheese Shops, Food, J...",4.0
2,TATCbqRSFNkmC_xNvNmmUA,Yutaka,"Specialty Food, Japanese, Ethnic Food, Restaur...",4.0
3,TKKX9bllRBMzp0NYr441PQ,Mi'Hito Sushi Laboratory,"Food, Poke, Fast Food, Sushi Bars, Restaurants...",4.0
4,bsFZnc2mYyGy5cFf5EWTWg,Aji Sai Japanese Restaurant,"Japanese, Restaurants, Food, Buffets",3.0
5,c93AzlUc_j0C-lkWBG92Sw,Sushi Shop,"Food, Japanese, Restaurants, Specialty Food, E...",4.5
6,hXVaXtDO9NnvCOSMtFuHRg,Kawa Sushi,"Food, Food Delivery Services, Restaurants, Jap...",3.5
7,jTZYireK6pnPb6Rni0jygw,Wow Sushi,"Food, Restaurants, Sushi Bars",4.0
8,kB1QyyvpQ5kgtIw1sYDfkQ,Top Sushi,"Food, Sushi Bars, Restaurants",2.0
9,q9_gLvTNf11etVxbH7JY0Q,Yasu,"Food, Japanese, Restaurants, Sushi Bars",4.5


In [65]:
recommendation.to_csv("sushi_loc1.csv",index=False)

In [66]:
# Scarborough Civic Centre
recommendation = BM25_recommend("Sushi", ReviewSummary, 43.77952047125285, -79.25780882879529)
recommendation

,business_id,name,categories,rating
0,G4pt2q9Tr5SkwCw3Wb43VA,Snow Time,"Shaved Snow, Food, Restaurants, Korean, Desser...",4.0
1,HTZFw4QDKhD8KRbnb5ZuJA,Chicken Kitchen,"Restaurants, Chicken Shop, Fast Food, Chicken ...",4.5
2,_xAJZOKBMPOe47p1MphB2w,Fishman Lobster Clubhouse Restaurant,"Seafood, Restaurants, Chinese, Live/Raw Food",4.0
3,a7mTbEi2N8Zd-r-8jlReww,Koryo Korean Barbeque,"Barbeque, Japanese, Korean, Fast Food, Restaur...",3.5
4,aAGo2MMC2hQlG5uhPPNnjA,Su&Shi Noodle Bar,"Restaurants, Noodles, Japanese, Food Court, Su...",3.0
5,kbUAnpn0QoDWRqFOA_oWMA,TO Food Fest,"Festivals, Food, Canadian (New), Restaurants, ...",4.0
6,m_rvCLB8j10NeUZPzuK1fA,Crown Pastries,"Bakeries, Food, Desserts, Middle Eastern, Rest...",4.5
7,r2qLNZaYTZKjvVXfXB1BLQ,The Bowl Noodle Bar,"Chinese, Restaurants, Tea Rooms, Food",3.0
8,z9_vbTmxItQ_qUn0mo-Nyw,Helious Cafe,"Cafes, Restaurants, Food, Juice Bars & Smoothi...",2.0
9,zu5dqKqPjJCycS5k-92Alg,Chaplus,"Waffles, Tea Rooms, Restaurants, Coffee & Tea,...",3.0


In [67]:
recommendation.to_csv("sushi_loc2.csv",index=False)

In [68]:
# Aga Khan Museum
recommendation = BM25_recommend("Sushi", ReviewSummary, 43.72803030610192, -79.32977026321979)
recommendation

,business_id,name,categories,rating
0,1jNteKQ2JuF6Sk1SI9X23Q,P F Chang's,"Food, Chinese, Restaurants, Asian Fusion, Vege...",3.0
1,5sKb4h7R5gdhePSflPD0Tw,Kiku,"Event Planning & Services, Caterers, Restauran...",2.5
2,EwHgw3rDJiqU3XDJvcpR8w,Subway,"Sandwiches, Restaurants, Fast Food",2.5
3,ExoqgA3aAz6bYidI0DSQvw,California Sandwiches,"Restaurants, Food, Sandwiches",4.0
4,FS8uGqtkLVl5P5iPhTAGjQ,Robin's,"Juice Bars & Smoothies, Restaurants, Cafes, Fo...",3.5
5,KOQF_A4XrRrWYBWl5aSZ7w,Chakula Hut,"Halal, African, Comfort Food, Restaurants, Ind...",3.5
6,Mksyt1TZQ-7vzCZjeuzfAg,CASA manila Restaurant,"Fast Food, Filipino, Restaurants, Chinese",3.0
7,m_rvCLB8j10NeUZPzuK1fA,Crown Pastries,"Bakeries, Food, Desserts, Middle Eastern, Rest...",4.5
8,yHCoJZh5nx0Onr5xgMaUHg,Galleria Supermarket,"Ethical Grocery, International Grocery, Korean...",4.0
9,zeJh7AoPgwtNNQSfl4eneQ,Granite Bistro,"Event Planning & Services, Caterers, American ...",5.0


In [69]:
recommendation.to_csv("sushi_loc3.csv",index=False)

### Bakery

In [70]:
# University of Toronto - St. George Campus:
recommendation = BM25_recommend("Bakery", ReviewSummary, 43.66311072883239, -79.39440813687946)
recommendation

,business_id,name,categories,rating
0,DlhhSd13HsuRqPT6h3kvAg,Harbord Bakery,"Kosher, Food, Bakeries, Restaurants",3.5
1,PGJd7yyMitAgYRRYvxxEHA,Venezia Bakery,"Restaurants, Bakeries, Food, Delis, Portuguese",4.0
2,R__NLOqjUlw8qf12KD-I5A,Ding Dong Pastries,"Food, Bakeries, Restaurants, Chinese",3.5
3,as5StqV9pbDhUM0AYP_j9g,Kim Moon Bakery,"Food, Restaurants, Chinese, Bakeries",3.5
4,eEKfOX6KtQmLgb8z-1VfUA,Riviera Bakery,"Cafes, Bakeries, Italian, Restaurants, Food",3.5
5,gbjIwB_r6y4YhaZMwEK6mw,Brick Street Bakery,"Sandwiches, Restaurants, Food, Bakeries",4.0
6,j2U2vs4WWlF6SR5SS_D1lA,Brazil Bakery and Pastry,"Restaurants, Bakeries, Brazilian, Food, Portug...",4.0
7,lkvHG6proAFP8_evKhdxKA,Chinese Bakery,"Bakeries, Food, Chinese, Restaurants",3.5
8,yGDiAVoQB8LX3OsJ4e2I0A,Kin-Kin Bakery & Bubble Tea,"Food, Chinese, Coffee & Tea, Restaurants, Bake...",3.5
9,yX4anPCzNGDQ-v8_uug2yg,Kensington Natural Bakery,"Desserts, Bakeries, Shopping, Food, Specialty ...",3.0


In [71]:
recommendation.to_csv("bakery_loc1.csv",index=False)

In [72]:
# Scarborough Civic Centre
recommendation = BM25_recommend("Bakery", ReviewSummary, 43.77952047125285, -79.25780882879529)
recommendation

,business_id,name,categories,rating
0,4J5WT8HM2IINOmVtvRihbg,Kin Kin Bubble Tea,"Cafes, Bakeries, Food, Restaurants",3.0
1,CINuanTB3t119JypJq0DTw,Lebanese Bakery,"Desserts, Food, Restaurants, Middle Eastern, B...",3.0
2,Koigz4oP9YnHlPhBhqXCxA,New Dynasty Bakery,"Restaurants, Food, Chinese, Bakeries",2.0
3,dtn4CT4bvZYYfqhGfXpRaw,Cosmos Agora,"Grocery, Bakeries, Specialty Food, Restaurants...",4.5
4,g7EMn2tfrmAKBW9rQQeeEw,Ladan Pastry & Bakery,"Restaurants, Persian/Iranian, Food, Bakeries",4.0
5,mUUTJY9_BqmAYxypHiqWrw,Francesca,"Food, Bakeries, Desserts, Italian, Coffee & Te...",4.0
6,m_rvCLB8j10NeUZPzuK1fA,Crown Pastries,"Bakeries, Food, Desserts, Middle Eastern, Rest...",4.5
7,rr9sk5DxL8wHxqIF48uJlg,Patisserie Royale,"Restaurants, Bakeries, Food, Middle Eastern",4.5
8,sA-YqTMe2bJmg1MgbfowuA,Whisked Gluten-Free,"Bakeries, Desserts, Food, Restaurants, Gluten-...",4.5
9,upbxmoNqlovW21O7bibV4Q,Aroma Bakery,"Chinese, Restaurants, Bakeries, Dim Sum, Food",4.5


In [73]:
recommendation.to_csv("bakery_loc2.csv",index=False)

In [74]:
# Aga Khan Museum
recommendation = BM25_recommend("Bakery", ReviewSummary, 43.72803030610192, -79.32977026321979)
recommendation

,business_id,name,categories,rating
0,5zM4mYdaRmchdQeo8dVw3g,C&C Supermarket,"Specialty Food, Restaurants, Food, Seafood Mar...",4.0
1,MTNDAZ5PrnflhmdyKqVJ8w,Kashgar Kebab House,"Middle Eastern, Restaurants, Food, Bakeries, M...",4.0
2,OIcneOOA3uhODls7ANBsnQ,Artisano Bakery Cafe,"Breakfast & Brunch, Restaurants, Sandwiches, F...",3.5
3,XglMkgLbu2U0wsbLTOQhxw,Tim Hortons,"Donuts, Breakfast & Brunch, Food, Restaurants,...",3.5
4,dtn4CT4bvZYYfqhGfXpRaw,Cosmos Agora,"Grocery, Bakeries, Specialty Food, Restaurants...",4.5
5,fwXKszT6j_f6pf0tp3MHLw,Patisserie Royale,"Bakeries, Food, Middle Eastern, Restaurants",5.0
6,g7EMn2tfrmAKBW9rQQeeEw,Ladan Pastry & Bakery,"Restaurants, Persian/Iranian, Food, Bakeries",4.0
7,isdHX5ykjTKfPNu6md2lkQ,Kabul Restaurant and Bakery,"Afghan, Mediterranean, Food, Ice Cream & Froze...",4.0
8,m_rvCLB8j10NeUZPzuK1fA,Crown Pastries,"Bakeries, Food, Desserts, Middle Eastern, Rest...",4.5
9,yHCoJZh5nx0Onr5xgMaUHg,Galleria Supermarket,"Ethical Grocery, International Grocery, Korean...",4.0


In [75]:
recommendation.to_csv("bakery_loc3.csv",index=False)

### Bar

In [76]:
# University of Toronto - St. George Campus:
recommendation = BM25_recommend("Bar", ReviewSummary, 43.66311072883239, -79.39440813687946)
recommendation

,business_id,name,categories,rating
0,4SyAKQevPuB4punANp41-Q,El Rey Mezcal Bar,"Nightlife, Specialty Food, Mexican, Food, Rest...",3.5
1,9ooTxMSL0xe1b0RvwTyDgg,Coffee Oysters Champagne,"Nightlife, Cocktail Bars, Tapas Bars, Food, Re...",4.0
2,Dgzrb9DTWozUcYOe4L9_FQ,Squirly's,"Comfort Food, Nightlife, Restaurants, Chicken ...",3.5
3,Qa4eXuZ1IFPwnVXJcpZWtw,Real Sports,"Canadian (New), Nightlife, Bars, Sports Bars, ...",3.5
4,jF3sWSqa-6TyROZgSUGrIw,Rock 'n' Horse Saloon,"Food, Pizza, Soul Food, Pubs, Comfort Food, Bu...",3.0
5,mISfQv_it_TCXpFf4BFzaw,Bar Volo,"Brasseries, Bars, Food, Beer, Wine & Spirits, ...",4.0
6,mQuDZwxxPe54MY4TT8lEPQ,Bar Fancy,"Nightlife, Restaurants, Bars, Chicken Shop, Co...",4.0
7,q7MorRPzU_J-iekeDKUKgw,Reposado Bar & Lounge,"Restaurants, Food, Jazz & Blues, Cocktail Bars...",4.0
8,v4y-YUuejiEdQNWD3QTJDw,Corks,"Food, Beer, Wine & Spirits, Tapas/Small Plates...",4.0
9,vOQ37XBxWePPfS5HJ7SoHw,Fox & Fiddle,"Comfort Food, Restaurants, Bars, Pubs, Nightlife",2.5


In [77]:
recommendation.to_csv("bar_loc1.csv",index=False)

In [78]:
# Scarborough Civic Centre
recommendation = BM25_recommend("Bar", ReviewSummary, 43.77952047125285, -79.25780882879529)
recommendation

,business_id,name,categories,rating
0,A0iy8p1nlL4Jt0Vssss3yQ,Supreme Restaurant & Bar,"Restaurants, Chinese, Soul Food, Nightlife, Ba...",3.0
1,C4X26YL0fNu5CuW8Flyo0g,Crème et Miel,"Restaurants, Juice Bars & Smoothies, Lebanese,...",4.5
2,GKEl7LNGUqJiQWNQ0vpH8Q,Milestones Restaurants,"American (Traditional), Bars, Restaurants, Com...",3.0
3,KvfaPsLlpOZAF276Lpvm4w,Tealicious Tea House,"Chinese, Taiwanese, Coffee & Tea, Bubble Tea, ...",3.0
4,WdhQvQbGd2HAiZ2UmlcVGA,Wild Wing,"Restaurants, Food, Chicken Wings, Barbeque, Am...",3.5
5,XNdeQD9nQYHBuNL4wIA9Kw,Daan Go Cafe,"Breakfast & Brunch, Bakeries, Food, Restaurant...",4.0
6,cWPTOM_o5toaXpq0gNxD7Q,Vanta Coffee Club & Eatery,"Food, Cafes, Coffee & Tea, Restaurants, Sandwi...",4.0
7,d3JhScywFHns4gdR7vIqlA,The Wing Spot,"Restaurants, Chicken Wings, American (Traditio...",4.0
8,r2qLNZaYTZKjvVXfXB1BLQ,The Bowl Noodle Bar,"Chinese, Restaurants, Tea Rooms, Food",3.0
9,z9_vbTmxItQ_qUn0mo-Nyw,Helious Cafe,"Cafes, Restaurants, Food, Juice Bars & Smoothi...",2.0


In [79]:
recommendation.to_csv("bar_loc2.csv",index=False)

In [80]:
# Aga Khan Museum
recommendation = BM25_recommend("Bar", ReviewSummary, 43.72803030610192, -79.32977026321979)
recommendation

,business_id,name,categories,rating
0,2PZRu5q70e0p7jEMxaXsDw,The Bier Markt,"Gastropubs, Belgian, Restaurants, Canadian (Ne...",3.0
1,8aoZuNwB2w0YmI887kky9g,TBG Garden Cafe,"Cafes, Food, Restaurants, Coffee & Tea",2.0
2,C4X26YL0fNu5CuW8Flyo0g,Crème et Miel,"Restaurants, Juice Bars & Smoothies, Lebanese,...",4.5
3,GsCd36irPdtQytfvjrxzEw,Adam's Halal Pub,"Fast Food, Burgers, Chicken Wings, Restaurants",3.5
4,Njc2gHb5Z_ufkA_ow7hKqw,Amsterdam Barrel House,"Food, Breweries, Canadian (New), Restaurants, ...",3.5
5,URvh6DgFyA-d6Z7OVKxczg,Rally Sports Bar + Smokehouse,"Smokehouse, Food, Barbeque, Nightlife, Comfort...",4.0
6,bCzqTllyOJoH3S7e_AHJNw,The Moose And Barrel Pub & Restaurant,"Nightlife, Restaurants, Food, Beer, Wine & Spi...",2.5
7,gHmkE8VoNjJZDLVdTpJPOw,Taylor's Landing Bar Grill Hub,"Nightlife, American (Traditional), Bars, Canad...",3.0
8,iQDPdxmXXvW-mgy9gsKXhQ,Starbucks Reserve,"Restaurants, Coffee Roasteries, Coffee & Tea, ...",4.0
9,pnStRO16rw-jXa9S2FQoEg,Alexandria Cafe,"Donairs, Middle Eastern, Food, Restaurants, Ho...",2.0


In [81]:
recommendation.to_csv("bar_loc3.csv",index=False)

### Steak House

In [82]:
# University of Toronto - St. George Campus:
recommendation = BM25_recommend("steak house", ReviewSummary, 43.66311072883239, -79.39440813687946)
recommendation

,business_id,name,categories,rating
0,27_N5P9HZ86IviONPT_twg,Woodlot Restaurant,"Comfort Food, Canadian (New), American (New), ...",4.0
1,DIVF4oo86EGupm5CC60sZg,Hunters Landing Bar Grill Hub Restaurant,"Restaurants, Nightlife, Bars, Comfort Food, Ca...",3.5
2,KT2E2HQtb0HA0zsjbVp2BQ,STK Toronto,"Cocktail Bars, Bars, American (Traditional), D...",3.0
3,T4g4k2OhCTsiqcP-ZXElHQ,Jules Bistro,"Restaurants, Food, French",4.0
4,WD2lLI59TpoTQRg4QFNSbg,Gdous Juicy Chicken House,"Bubble Tea, Restaurants, Food, Chicken Wings, ...",3.5
5,_DLxHAqZtGCeNFp6a692Cg,Rose City Kitchen,"Sandwiches, Salad, Restaurants, Falafel, Middl...",3.0
6,_cjPEH9wXhKS-HQe_U3M4Q,Le Sélect Bistro,"French, Restaurants, Food, Breakfast & Brunch,...",4.0
7,dd3GZs6zW2t7BM0cbxAzvg,La Palette,"Restaurants, French, Nightlife, Desserts, Beer...",4.0
8,ivbMuKOwwX2B70gFggzT-g,Canyon Creek,"Seafood, Steakhouses, Restaurants, Food, Beer,...",3.5
9,qflIO8MUjwBnpY0R9wZ9Nw,Cumbraes,"Butcher, Food, Sandwiches, Restaurants, Specia...",4.0


In [83]:
recommendation.to_csv("sh_loc1.csv",index=False)

In [84]:
# Scarborough Civic Centre
recommendation = BM25_recommend("steak house", ReviewSummary, 43.77952047125285, -79.25780882879529)
recommendation

,business_id,name,categories,rating
0,1PnHQEOXnxosKYhB86Ou7A,Freshii,"Salad, Restaurants, Fast Food, Food, Poke, Veg...",3.0
1,6j67nog58__bFPqd620Rrw,JJM Dbarkadz Filipino Cuisine,"Food, Restaurants, Filipino, Food Delivery Ser...",4.0
2,AIasveRX0245WeAH0C46Uw,Chick Fiesta,"Chicken Shop, South African, African, Portugue...",4.0
3,GKEl7LNGUqJiQWNQ0vpH8Q,Milestones Restaurants,"American (Traditional), Bars, Restaurants, Com...",3.0
4,JTIbsAs3qUq-0bNIh404ng,City Fried Chicken,"Restaurants, Fast Food, Fish & Chips, Cheesest...",4.0
5,VnvRZia9M18-IUarfdnYfA,Mucho Burrito,"Mexican, Food, Restaurants, Tex-Mex",3.0
6,Xjcd_--LJYBlkdLSXhbR9g,Mami's Philippine Cuisine,"Fast Food, Restaurants, Filipino",3.5
7,a7mTbEi2N8Zd-r-8jlReww,Koryo Korean Barbeque,"Barbeque, Japanese, Korean, Fast Food, Restaur...",3.5
8,iFWgJ0zmVJ27IeyHcdm1kw,Nugget Pizza and Wings,"Burgers, Chicken Wings, Fast Food, Restaurants...",4.0
9,sHIJktNJTTJPOGhzXMI2Fg,Restaurant Fifty Nine,"Restaurants, Fast Food, Chinese",3.0


In [85]:
recommendation.to_csv("sh_loc2.csv",index=False)

In [86]:
# Aga Khan Museum
recommendation = BM25_recommend("steak house", ReviewSummary, 43.72803030610192, -79.32977026321979)
recommendation

,business_id,name,categories,rating
0,2PZRu5q70e0p7jEMxaXsDw,The Bier Markt,"Gastropubs, Belgian, Restaurants, Canadian (Ne...",3.0
1,3Z55ll2Rx43I96aozFG3Eg,Pizza Land,"Restaurants, Fast Food, Pizza",3.5
2,AY4WY_rfrQum3Ad_ohNhVw,Sultan of Samosas,"Food, Restaurants, Indian, Caterers, Event Pla...",4.0
3,CyKi4WgsRNpp3jgsxBEcDg,North Shore Pie,"Bakeries, Restaurants, Food, Cafes",4.0
4,ExoqgA3aAz6bYidI0DSQvw,California Sandwiches,"Restaurants, Food, Sandwiches",4.0
5,GZ12skjXwuMuH3LidRN-mA,Chipotle Mexican Grill,"Restaurants, Spanish, Mexican, Fast Food",3.5
6,LG7-BUvDLoOZCHDjyiBQFQ,Dirty Grill,"Salad, Restaurants, Juice Bars & Smoothies, Fo...",4.0
7,OIcneOOA3uhODls7ANBsnQ,Artisano Bakery Cafe,"Breakfast & Brunch, Restaurants, Sandwiches, F...",3.5
8,fbBTvnuHPMsAQs_RShdVyw,Grilltime Gourmet Meat Shop,"Specialty Food, Food, Meat Shops, Grocery, Res...",4.0
9,gHmkE8VoNjJZDLVdTpJPOw,Taylor's Landing Bar Grill Hub,"Nightlife, American (Traditional), Bars, Canad...",3.0


In [87]:
recommendation.to_csv("sh_loc3.csv",index=False)

### Chinese Food

In [88]:
# University of Toronto - St. George Campus:
recommendation = BM25_recommend("Chinese Food", ReviewSummary, 43.66311072883239, -79.39440813687946)
recommendation

,business_id,name,categories,rating
0,C1JNZrYl3H-YzF1dys9SoQ,Spadina Garden Restaurant,"Indian, Asian Fusion, Specialty Food, Chinese,...",3.0
1,FcNtrDGaWmkDzlHaq9F1UQ,Szechuan Szechuan Fine Chinese Cuisine,"Restaurants, Specialty Food, Food, Ethnic Food...",3.0
2,J_btDyZbIv0hZNjrw56zlA,New Sky Restaurant,"Restaurants, Chinese, Peruvian, Specialty Food...",3.5
3,OllK5_S-7svgSwbUfx1xYA,King's Noodle Restaurant,"Restaurants, Chinese, Imported Food, Barbeque,...",3.5
4,SxjNgkzAlUG-wt2rSbc2HQ,New Ho King,"Chinese, Restaurants, Food, Specialty Food, Et...",2.5
5,XGc_QdKg9oP8bZ3VM1e5IA,Good View Restaurant,"Chinese, Food, Restaurants, Ethnic Food, Noodl...",3.5
6,f-ookZOX4pBNldaOHMkJ4A,Hong Shing Chinese Restaurant,"Food Delivery Services, Food, Chinese, Restaur...",3.0
7,hqpPDu238T1tpBa64Q0TSw,Tasty Chinese Food,"Restaurants, Chinese, Ethnic Food, Specialty F...",3.0
8,mt5kFJr1rOqIajMguELR9Q,Saigon Flower Restaurant,"Imported Food, Chinese, Food, Ethnic Food, Vie...",2.5
9,xdfGzrJ1mx3mBx6m14Ebhw,Goldstone Noodle Restaurant,"Restaurants, Chinese, Noodles, Ethnic Food, Fo...",3.0


In [89]:
recommendation.to_csv("cf_loc1.csv",index=False)

In [90]:
# Scarborough Civic Centre
recommendation = BM25_recommend("Chinese Food", ReviewSummary, 43.77952047125285, -79.25780882879529)
recommendation

,business_id,name,categories,rating
0,0gj8bpTF6I6T_C5xyBUafw,Arden Fast Food,"Chinese, Food, Restaurants",3.0
1,1kqLZp0_FEMHGx-W4TjTLQ,Hong Kong Style Tea & Fast Food,"Fast Food, Restaurants, Chinese, Food Court",4.0
2,B8jsJJMhbASM2IvkU4tmKg,The Rice Kitchen,"Food Delivery Services, Restaurants, Food, Chi...",3.5
3,HakulcKdPE-RaWCVF1GUKQ,Aberdeen Delicatessen,"Chinese, Desserts, Food, Restaurants",2.0
4,Qzq-m_TKJ-Iv1BmYLcjlVA,Spicy Dragon,"Restaurants, Food Delivery Services, Chinese, ...",3.5
5,_xAJZOKBMPOe47p1MphB2w,Fishman Lobster Clubhouse Restaurant,"Seafood, Restaurants, Chinese, Live/Raw Food",4.0
6,dTWfATVrBfKj7Vdn0qWVWg,Flavor Cuisine,"Food Court, Chinese, Restaurants",2.5
7,qftPq-opqmbs1BKtRj3SgA,Sure Win Cafe,"Chinese, Food Court, Restaurants, Cafes",3.0
8,r2qLNZaYTZKjvVXfXB1BLQ,The Bowl Noodle Bar,"Chinese, Restaurants, Tea Rooms, Food",3.0
9,sHIJktNJTTJPOGhzXMI2Fg,Restaurant Fifty Nine,"Restaurants, Fast Food, Chinese",3.0


In [91]:
recommendation.to_csv("cf_loc2.csv",index=False)

In [92]:
# Aga Khan Museum
recommendation = BM25_recommend("Chinese Food", ReviewSummary, 43.72803030610192, -79.32977026321979)
recommendation

,business_id,name,categories,rating
0,1jNteKQ2JuF6Sk1SI9X23Q,P F Chang's,"Food, Chinese, Restaurants, Asian Fusion, Vege...",3.0
1,5zM4mYdaRmchdQeo8dVw3g,C&C Supermarket,"Specialty Food, Restaurants, Food, Seafood Mar...",4.0
2,6U73zDP4NuFKyOQLSigcPA,Spring China House,"Restaurants, Food Delivery Services, Food, Chi...",3.0
3,9xPK9u_AGM1sxoG2CU9psA,Golden Wok Chinese Restaurant,"Chinese, Food, Food Delivery Services, Restaur...",3.5
4,GoOkCxPFM5i5lxHgFQMj1w,Asian Legend,"Chinese, Restaurants, Food",3.0
5,PMPMHHLe9vFzcR67iCjgLA,Faryab Kabab,"Lebanese, Kebab, Persian/Iranian, Middle Easte...",4.0
6,ZmnCN-U7mhuZtlWHRsSdtA,Heirloom,"Food, Food Trucks, Restaurants, Burgers, Event...",4.0
7,bwFxrxHrz9I36awGc-yjjw,TAO Northern Chinese Cuisine,"Dim Sum, Seafood, Chinese, Noodles, Food, Rest...",3.5
8,nGCKQlzat6nEKTImlpg_hg,Joy Fortune Chinese Cuisine,"Beer, Wine & Spirits, Food, Chinese, Restaurants",3.5
9,yHCoJZh5nx0Onr5xgMaUHg,Galleria Supermarket,"Ethical Grocery, International Grocery, Korean...",4.0


In [93]:
recommendation.to_csv("cf_loc3.csv",index=False)

### Indian Food

In [94]:
# University of Toronto - St. George Campus:
recommendation = BM25_recommend("Indian Food", ReviewSummary, 43.66311072883239, -79.39440813687946)
recommendation

,business_id,name,categories,rating
0,8td1xV3B0hJc3wYpGmjtRQ,Butter Chicken Factory,"Indian, Restaurants, Food Delivery Services, F...",3.5
1,CkncJJ9tKgsGULXTTKmfdw,309 Dhaba Indian Excellence,"Indian, Restaurants, Pakistani, Venues & Event...",3.0
2,Ijh1AcbH-F3CyhYT_5qL8Q,Indian Grill,"Food, Restaurants, Indian",4.0
3,J9BmILDpV1Pr3GKU9XhjTQ,Little India Restaurant,"Buffets, Pakistani, Vegetarian, Indian, Restau...",3.5
4,Pz13Ru_-U4qoZiu89ezVmQ,Utsav,"Food Delivery Services, Food, Restaurants, Indian",4.0
5,aXnWWp6Reaep4-0iKWlYHQ,TKRE - The Kathi Roll Express,"Indian, Restaurants, Asian Fusion, Food, Juice...",3.5
6,bfR-vJvrjdOJaWsXGJgzPA,Veda Healthy Indian Takeout,"Restaurants, Pakistani, Indian, Fast Food, Foo...",2.5
7,nAnpTnodDchSl9H0f6R2pw,Bhoj Indian Cuisine,"Event Planning & Services, Caterers, Food Deli...",4.0
8,nYKBZdzoA-kjqiK08IPSiA,Maroli Indian Restaurant,"Pakistani, Diners, Event Planning & Services, ...",3.5
9,sJ0MYSAIVK28cMzh-s-NPA,Amaya Express,"Fast Food, Indian, Restaurants",2.5


In [95]:
recommendation.to_csv("If_loc1.csv",index=False)

In [96]:
# Scarborough Civic Centre
recommendation = BM25_recommend("Indian Food", ReviewSummary, 43.77952047125285, -79.25780882879529)
recommendation

,business_id,name,categories,rating
0,2vyjdKZJM5fv0v7Iob0FcQ,Mirchi Dhaba,"Food, Imported Food, Do-It-Yourself Food, Spec...",3.0
1,A0iy8p1nlL4Jt0Vssss3yQ,Supreme Restaurant & Bar,"Restaurants, Chinese, Soul Food, Nightlife, Ba...",3.0
2,C_chktl5jFeRqu1C0rmScA,Kairali - Taste of Kerala,"Restaurants, Indian, Food",3.5
3,Exsi3PIwpfvuoyF8BfGYIQ,HaveMore,"Food Court, Indian, Caterers, Event Planning &...",4.0
4,IfmSqAJ6_IeE1ffQ2AcxsQ,Best Kabab and Karahi,"Indian, Pakistani, Restaurants, Food",3.5
5,Qzq-m_TKJ-Iv1BmYLcjlVA,Spicy Dragon,"Restaurants, Food Delivery Services, Chinese, ...",3.5
6,YLeAUkFFJri_21uTkyrVFg,Malaysian Garam Masala,"Food, Restaurants, Indian, Malaysian",4.0
7,ZLPPaAJnEvN8311zwXvw6w,Touch Down Foods,"Desserts, Grocery, Restaurants, Imported Food,...",5.0
8,lz9oUUVaPJsC5aaU_6nzsQ,Fresh Choice Cafe,"Restaurants, Coffee & Tea, Food, Cafes, Delis",4.5
9,my5v0UE6gOukDVmWQB5RpA,Golden Spoon,"Food Delivery Services, Restaurants, Food, Pak...",1.0


In [97]:
recommendation.to_csv("If_loc2.csv",index=False)

In [98]:
# Aga Khan Museum
recommendation = BM25_recommend("Indian Food", ReviewSummary, 43.72803030610192, -79.32977026321979)
recommendation

,business_id,name,categories,rating
0,-J6FVdY9pSgAdFmmalO-pQ,Jatujak,"Food, Thai, Food Delivery Services, Restaurants",4.0
1,1jNteKQ2JuF6Sk1SI9X23Q,P F Chang's,"Food, Chinese, Restaurants, Asian Fusion, Vege...",3.0
2,AY4WY_rfrQum3Ad_ohNhVw,Sultan of Samosas,"Food, Restaurants, Indian, Caterers, Event Pla...",4.0
3,GZ12skjXwuMuH3LidRN-mA,Chipotle Mexican Grill,"Restaurants, Spanish, Mexican, Fast Food",3.5
4,KOQF_A4XrRrWYBWl5aSZ7w,Chakula Hut,"Halal, African, Comfort Food, Restaurants, Ind...",3.5
5,MTNDAZ5PrnflhmdyKqVJ8w,Kashgar Kebab House,"Middle Eastern, Restaurants, Food, Bakeries, M...",4.0
6,URvh6DgFyA-d6Z7OVKxczg,Rally Sports Bar + Smokehouse,"Smokehouse, Food, Barbeque, Nightlife, Comfort...",4.0
7,WOjGXrfQnfVEVwbMxzhu8A,Popeyes Louisiana Kitchen,"American (Traditional), Fast Food, Chicken Win...",3.0
8,YIez_A3WOt9J2SXN7OMa2Q,Allwyn's Bakery,"Restaurants, Bakeries, Caribbean, Food",4.0
9,oo73pe5tERpDxaa_eSY2tQ,Iqbal Kebab & Sweet Centre,"Restaurants, Barbeque, Grocery, Seafood, Pakis...",4.0


In [99]:
recommendation.to_csv("If_loc3.csv",index=False)

### Bubble Tea

In [100]:
# University of Toronto - St. George Campus:
recommendation = BM25_recommend("Bubble Tea", ReviewSummary, 43.66311072883239, -79.39440813687946)
recommendation

,business_id,name,categories,rating
0,0fhaHYT_NR2_qB3DIC2i_Q,CoCo Fresh Tea & Juice,"Food, Bubble Tea, Juice Bars & Smoothies, Tea ...",4.0
1,JLTxzBEW4IrhraaonG-vUw,Nohohon Tea - Queen West,"Japanese, Tea Rooms, Food, Vegan, Cafes, Bubbl...",4.0
2,KnV-IaFLBNR4aT7c_voXjQ,Chatime,"Coffee & Tea, Restaurants, Taiwanese, Food, Ca...",3.5
3,YHXczxm4W3BkGT-z7vZBBw,Chatime - Yonge & College,"Tea Rooms, Taiwanese, Food, Chinese, Coffee & ...",3.5
4,ZDDzKXN_JXe7694zePRuTQ,Chatime - Dundas,"Chinese, Restaurants, Taiwanese, Tea Rooms, Et...",3.5
5,_PqvLxLW1uZJlJZzGMWErQ,Bubble Republic on Bay,"Food, Tea Rooms, Taiwanese, Restaurants",4.0
6,i6gKYG_YFuF5o90jPuWYvw,CoCo Fresh Juice & Tea,"Food, Juice Bars & Smoothies, Cafes, Tea Rooms...",4.0
7,mlHwA-JpSi7hU3xnEu7aQw,Hi Tea,"Comfort Food, Food, Desserts, Coffee & Tea, Re...",2.5
8,pRwh3BEM-dhixAGJl_TgUA,Ten Ren Tea Company,"Restaurants, Chinese, Coffee & Tea, Food",2.5
9,zFR99jgMi-qzaJXIx8MXHA,Labothéry,"Bubble Tea, Restaurants, Cafes, Food",3.5


In [101]:
recommendation.to_csv("bb_loc1.csv",index=False)

In [102]:
# Scarborough Civic Centre
recommendation = BM25_recommend("Bubble Tea", ReviewSummary, 43.77952047125285, -79.25780882879529)
recommendation

,business_id,name,categories,rating
0,F09jtEb9KEph2L7Pbji5QQ,Golden Bubbles,"Ice Cream & Frozen Yogurt, Waffles, Desserts, ...",3.5
1,KvfaPsLlpOZAF276Lpvm4w,Tealicious Tea House,"Chinese, Taiwanese, Coffee & Tea, Bubble Tea, ...",3.0
2,LN2X2X2PVsd56tqLIEku9Q,Chill,"Restaurants, Coffee & Tea, Food, Chinese, Dess...",4.0
3,NR2-lpZ91nDvP5pfavhtIQ,Tea A Break Tea House,"Chinese, Restaurants, Food, Tea Rooms",2.5
4,Q6cgDdRy8BCmw745WZJCxA,Twilight Cafe,"Bubble Tea, Cafes, Social Clubs, Restaurants, ...",3.0
5,XNdeQD9nQYHBuNL4wIA9Kw,Daan Go Cafe,"Breakfast & Brunch, Bakeries, Food, Restaurant...",4.0
6,ZPI1t-WcZruILq7OQYnZHg,Real Fruit Bubble Tea,"Juice Bars & Smoothies, Food, Tea Rooms, Asian...",4.5
7,kBYBXh7M9wKXvHgluJWiLQ,Annie's Tea House,"Food Stands, Juice Bars & Smoothies, Bubble Te...",4.0
8,r2qLNZaYTZKjvVXfXB1BLQ,The Bowl Noodle Bar,"Chinese, Restaurants, Tea Rooms, Food",3.0
9,zu5dqKqPjJCycS5k-92Alg,Chaplus,"Waffles, Tea Rooms, Restaurants, Coffee & Tea,...",3.0


In [103]:
recommendation.to_csv("bb_loc2.csv",index=False)

In [104]:
# Aga Khan Museum
recommendation = BM25_recommend("Bubble Tea", ReviewSummary, 43.72803030610192, -79.32977026321979)
recommendation

,business_id,name,categories,rating
0,-J6FVdY9pSgAdFmmalO-pQ,Jatujak,"Food, Thai, Food Delivery Services, Restaurants",4.0
1,1hUvpPDLiVS7Ht2LPROFTA,Demetres Shops At Don Mills,"Cafes, Coffee & Tea, Desserts, Food, Ice Cream...",4.0
2,5CSCTpBZYl_uDN7ZjSp5yQ,Real Fruit Bubble Tea,"Food Court, Restaurants, Food, Juice Bars & Sm...",2.5
3,5zM4mYdaRmchdQeo8dVw3g,C&C Supermarket,"Specialty Food, Restaurants, Food, Seafood Mar...",4.0
4,CyKi4WgsRNpp3jgsxBEcDg,North Shore Pie,"Bakeries, Restaurants, Food, Cafes",4.0
5,Mksyt1TZQ-7vzCZjeuzfAg,CASA manila Restaurant,"Fast Food, Filipino, Restaurants, Chinese",3.0
6,Njc2gHb5Z_ufkA_ow7hKqw,Amsterdam Barrel House,"Food, Breweries, Canadian (New), Restaurants, ...",3.5
7,iQDPdxmXXvW-mgy9gsKXhQ,Starbucks Reserve,"Restaurants, Coffee Roasteries, Coffee & Tea, ...",4.0
8,vOdY5xomz8nNX756zNYQxQ,Real Fruit Bubble Tea,"Restaurants, Taiwanese, Food, Bubble Tea, Coff...",2.5
9,yHCoJZh5nx0Onr5xgMaUHg,Galleria Supermarket,"Ethical Grocery, International Grocery, Korean...",4.0


In [105]:
recommendation.to_csv("bb_loc3.csv",index=False)

### Mexican Food

In [106]:
# University of Toronto - St. George Campus:
recommendation = BM25_recommend("Mexican Food", ReviewSummary, 43.66311072883239, -79.39440813687946)
recommendation

,business_id,name,categories,rating
0,1W_gZM_uuEvJqapbIl6z9Q,El Charro,"Food Stands, Food, Mexican, Street Vendors, Re...",4.0
1,Io1WmI1cUc8QaUVxSx1KuA,Perola Supermarket,"Restaurants, Food, Grocery, Latin American",4.0
2,O_UC_izJXcAmkm6HlEyGSA,Playa Cabana,"Food, Nightlife, Bars, Beer, Wine & Spirits, R...",4.0
3,PsNvBNvoP5kq4yjjTlYhXw,Agave Y Aguacate,"Mexican, Food, Ethnic Food, Specialty Food, Re...",4.0
4,VRwT0pscR5vESCrAnUpNwQ,Wilbur Mexicana,"Restaurants, Ethnic Food, Mexican, Food, Speci...",4.0
5,hJda5g5v_2rT3D-3mSf7cQ,Tequila Sun Rise Cantina & Grill,"Restaurants, Mexican, Food, Specialty Food",2.5
6,mES9E5K4HbYoPFXGv0pUuw,Como En Casa,"Food, Restaurants, Ethnic Food, Specialty Food...",4.0
7,n_4PTgvjsKc74u8AwXOfAQ,Mexican Salsas,"Gluten-Free, Restaurants, Fast Food, Mexican",4.0
8,sPPh1LJllyr5jUpKFTxxag,Asada Mexican Grill,"Mexican, Specialty Food, Restaurants, Food, Et...",4.0
9,uCDVOidGLpD-Q7_y82nx2g,Mexitaco,"Restaurants, Food, Mexican, Specialty Food",3.5


In [107]:
recommendation.to_csv("mf_loc1.csv",index=False)

In [108]:
# Scarborough Civic Centre
recommendation = BM25_recommend("Mexican Food", ReviewSummary, 43.77952047125285, -79.25780882879529)
recommendation

,business_id,name,categories,rating
0,6j67nog58__bFPqd620Rrw,JJM Dbarkadz Filipino Cuisine,"Food, Restaurants, Filipino, Food Delivery Ser...",4.0
1,F9tePBgROEAcd8xZqTZUfg,Jollibee,"Fast Food, Chicken Wings, Filipino, Restaurants",3.5
2,K2CdjES-IZDDEgetOnf0Vw,One2 Snacks,"Asian Fusion, Restaurants, Fast Food, Malaysia...",4.5
3,LwH0uzsl_Rsb5MsJqaNc6w,Fat Bastard Burrito,"Restaurants, Mexican, Fast Food",3.0
4,Qzq-m_TKJ-Iv1BmYLcjlVA,Spicy Dragon,"Restaurants, Food Delivery Services, Chinese, ...",3.5
5,URGUzMJRhK5gLQKC4UztTQ,Taco bell,"Fast Food, Mexican, Restaurants",3.0
6,VnvRZia9M18-IUarfdnYfA,Mucho Burrito,"Mexican, Food, Restaurants, Tex-Mex",3.0
7,Xjcd_--LJYBlkdLSXhbR9g,Mami's Philippine Cuisine,"Fast Food, Restaurants, Filipino",3.5
8,kbUAnpn0QoDWRqFOA_oWMA,TO Food Fest,"Festivals, Food, Canadian (New), Restaurants, ...",4.0
9,kiyTiW-2Ba65O3uAMaagTA,Mucho Burrito,"Mexican, Fast Food, Restaurants",2.5


In [109]:
recommendation.to_csv("mf_loc2.csv",index=False)

In [110]:
# Aga Khan Museum
recommendation = BM25_recommend("Mexican Food", ReviewSummary, 43.72803030610192, -79.32977026321979)
recommendation

,business_id,name,categories,rating
0,1jNteKQ2JuF6Sk1SI9X23Q,P F Chang's,"Food, Chinese, Restaurants, Asian Fusion, Vege...",3.0
1,3Z55ll2Rx43I96aozFG3Eg,Pizza Land,"Restaurants, Fast Food, Pizza",3.5
2,FuMc84iDRyPHD8tpkME0yA,Fat Bastard Burrito,"Restaurants, Fast Food, Mexican",2.5
3,GZ12skjXwuMuH3LidRN-mA,Chipotle Mexican Grill,"Restaurants, Spanish, Mexican, Fast Food",3.5
4,LG7-BUvDLoOZCHDjyiBQFQ,Dirty Grill,"Salad, Restaurants, Juice Bars & Smoothies, Fo...",4.0
5,Mksyt1TZQ-7vzCZjeuzfAg,CASA manila Restaurant,"Fast Food, Filipino, Restaurants, Chinese",3.0
6,OIcneOOA3uhODls7ANBsnQ,Artisano Bakery Cafe,"Breakfast & Brunch, Restaurants, Sandwiches, F...",3.5
7,URvh6DgFyA-d6Z7OVKxczg,Rally Sports Bar + Smokehouse,"Smokehouse, Food, Barbeque, Nightlife, Comfort...",4.0
8,gHmkE8VoNjJZDLVdTpJPOw,Taylor's Landing Bar Grill Hub,"Nightlife, American (Traditional), Bars, Canad...",3.0
9,yHCoJZh5nx0Onr5xgMaUHg,Galleria Supermarket,"Ethical Grocery, International Grocery, Korean...",4.0


In [111]:
recommendation.to_csv("mf_loc3.csv",index=False)

### Seafood

In [112]:
# University of Toronto - St. George Campus:
recommendation = BM25_recommend("seafood", ReviewSummary, 43.66311072883239, -79.39440813687946)
recommendation

,business_id,name,categories,rating
0,0Mhw1lD_Zwsv4Vo6oQrPWA,Wah Too Seafood Restaurant,"Food, Seafood, Restaurants, Chinese, Hakka",3.5
1,AQJPI5rnkb7pBnBMWtSjvw,The Shore Club - Toronto,"Seafood, Restaurants, Bars, Steakhouses, Night...",4.0
2,C7bK6Sd0LcvH7SOAjQu3LA,Roma1090,"Italian, Food, Specialty Food, Mediterranean, ...",3.5
3,EWmwbOm_4UhOtvLaBzHpPA,Fresh Off The Boat,"Fish & Chips, Specialty Food, Seafood, Food, S...",3.5
4,IFXL6wMLBgBSGSiLBjtViA,Celebrity Hotpot,"Hot Pot, Specialty Food, Restaurants, Food, Ch...",3.5
5,JEXasd9XUHFbt_fIdmdK8g,Restaurant La Banane,"Food, Seafood, Restaurants, Beer, Wine & Spiri...",4.0
6,J_btDyZbIv0hZNjrw56zlA,New Sky Restaurant,"Restaurants, Chinese, Peruvian, Specialty Food...",3.5
7,Qqm3yVcqTbC1IOKu0TG2nQ,Dal Moro's Fresh Pasta To Go,"Italian, Restaurants, Fast Food",3.5
8,nT16Y6AsJDwEpUB1JICKzg,St Lawrence Market,"Farmers Market, Sandwiches, Restaurants, Groce...",4.5
9,trKyIRyjKqVSZmcU0AnICQ,Pearl Diver,"Seafood, Restaurants, Irish, Breakfast & Brunc...",4.0


In [113]:
recommendation.to_csv("sf_loc1.csv",index=False)

In [114]:
# Scarborough Civic Centre
recommendation = BM25_recommend("seafood", ReviewSummary, 43.77952047125285, -79.25780882879529)
recommendation

,business_id,name,categories,rating
0,6j67nog58__bFPqd620Rrw,JJM Dbarkadz Filipino Cuisine,"Food, Restaurants, Filipino, Food Delivery Ser...",4.0
1,GKEl7LNGUqJiQWNQ0vpH8Q,Milestones Restaurants,"American (Traditional), Bars, Restaurants, Com...",3.0
2,HTZFw4QDKhD8KRbnb5ZuJA,Chicken Kitchen,"Restaurants, Chicken Shop, Fast Food, Chicken ...",4.5
3,NE596RKQomLvdrWiM9DYWg,Golden Jade Restaurant,"Chinese, Restaurants, Food",3.0
4,PCdQZQFvcjeiiXW_W0HBKA,Kenny's Noodles,"Ethnic Food, Chinese, Specialty Food, Food, Re...",4.5
5,YLeAUkFFJri_21uTkyrVFg,Malaysian Garam Masala,"Food, Restaurants, Indian, Malaysian",4.0
6,_xAJZOKBMPOe47p1MphB2w,Fishman Lobster Clubhouse Restaurant,"Seafood, Restaurants, Chinese, Live/Raw Food",4.0
7,aAGo2MMC2hQlG5uhPPNnjA,Su&Shi Noodle Bar,"Restaurants, Noodles, Japanese, Food Court, Su...",3.0
8,eSSr6I0nt-IZrkq4RgrUbA,Yumys Chicken & Seafood,"Restaurants, Fast Food, Chicken Shop, Seafood",2.0
9,r2qLNZaYTZKjvVXfXB1BLQ,The Bowl Noodle Bar,"Chinese, Restaurants, Tea Rooms, Food",3.0


In [115]:
recommendation.to_csv("sf_loc2.csv",index=False)

In [116]:
# Aga Khan Museum
recommendation = BM25_recommend("seafood", ReviewSummary, 43.72803030610192, -79.32977026321979)
recommendation

,business_id,name,categories,rating
0,2PZRu5q70e0p7jEMxaXsDw,The Bier Markt,"Gastropubs, Belgian, Restaurants, Canadian (Ne...",3.0
1,5sKb4h7R5gdhePSflPD0Tw,Kiku,"Event Planning & Services, Caterers, Restauran...",2.5
2,5zM4mYdaRmchdQeo8dVw3g,C&C Supermarket,"Specialty Food, Restaurants, Food, Seafood Mar...",4.0
3,9xPK9u_AGM1sxoG2CU9psA,Golden Wok Chinese Restaurant,"Chinese, Food, Food Delivery Services, Restaur...",3.5
4,Mksyt1TZQ-7vzCZjeuzfAg,CASA manila Restaurant,"Fast Food, Filipino, Restaurants, Chinese",3.0
5,bwFxrxHrz9I36awGc-yjjw,TAO Northern Chinese Cuisine,"Dim Sum, Seafood, Chinese, Noodles, Food, Rest...",3.5
6,fbBTvnuHPMsAQs_RShdVyw,Grilltime Gourmet Meat Shop,"Specialty Food, Food, Meat Shops, Grocery, Res...",4.0
7,nGCKQlzat6nEKTImlpg_hg,Joy Fortune Chinese Cuisine,"Beer, Wine & Spirits, Food, Chinese, Restaurants",3.5
8,oo73pe5tERpDxaa_eSY2tQ,Iqbal Kebab & Sweet Centre,"Restaurants, Barbeque, Grocery, Seafood, Pakis...",4.0
9,yHCoJZh5nx0Onr5xgMaUHg,Galleria Supermarket,"Ethical Grocery, International Grocery, Korean...",4.0


In [117]:
recommendation.to_csv("sf_loc3.csv",index=False)